In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression_linear(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
def NN_model_structure_regression_relu(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('relu'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [20]:
def NN_model_structure_regression_sigmoid(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('sigmoid'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [21]:
def NN_model_structure_regression_tanh(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('tanh'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [22]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [23]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [24]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_linear(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 415us/step - 

1052/1052 [==============================] - 0s 94us/step - loss: 21.7028 - val_loss: 25.9798
Epoch 66/1000
1052/1052 [==============================] - 0s 88us/step - loss: 21.8965 - val_loss: 25.6126
Epoch 67/1000
1052/1052 [==============================] - 0s 91us/step - loss: 21.7988 - val_loss: 25.2726
Epoch 68/1000
1052/1052 [==============================] - 0s 91us/step - loss: 22.1205 - val_loss: 26.0335
Epoch 69/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.1537 - val_loss: 25.6905
Epoch 70/1000
1052/1052 [==============================] - 0s 94us/step - loss: 21.5384 - val_loss: 25.8591
Epoch 71/1000
1052/1052 [==============================] - 0s 94us/step - loss: 22.1216 - val_loss: 25.4433
Epoch 72/1000
1052/1052 [==============================] - 0s 88us/step - loss: 21.6371 - val_loss: 26.8279
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 21.7656 - val_loss: 27.8368
Epoch 74/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 97us/step - loss: 21.4289 - val_loss: 26.1980
Epoch 141/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.1139 - val_loss: 25.8013
Epoch 142/1000
1052/1052 [==============================] - 0s 87us/step - loss: 22.1739 - val_loss: 24.4682
Epoch 143/1000
1052/1052 [==============================] - 0s 88us/step - loss: 22.0249 - val_loss: 25.1431
Epoch 144/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.5485 - val_loss: 26.3245
Epoch 145/1000
1052/1052 [==============================] - 0s 92us/step - loss: 22.0413 - val_loss: 25.5209
Epoch 146/1000
1052/1052 [==============================] - 0s 92us/step - loss: 22.4831 - val_loss: 25.2396
Epoch 147/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.6379 - val_loss: 25.4674
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 21.8810 - val_loss: 25.0900
Epoch 149/1000
1052/1052 [========

1052/1052 [==============================] - 0s 91us/step - loss: 16.1224 - val_loss: 18.4178
Epoch 216/1000
1052/1052 [==============================] - 0s 86us/step - loss: 15.6449 - val_loss: 18.2166
Epoch 217/1000
1052/1052 [==============================] - 0s 92us/step - loss: 16.2917 - val_loss: 18.2469
Epoch 218/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.1032 - val_loss: 18.7159
Epoch 219/1000
1052/1052 [==============================] - 0s 87us/step - loss: 15.4412 - val_loss: 18.1310
Epoch 220/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.2024 - val_loss: 19.0627
Epoch 221/1000
1052/1052 [==============================] - 0s 87us/step - loss: 15.4895 - val_loss: 17.8591
Epoch 222/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.6981 - val_loss: 20.0387
Epoch 223/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.4380 - val_loss: 19.3242
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 10.9365 - val_loss: 13.8333
Epoch 291/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.2199 - val_loss: 13.2054
Epoch 292/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.6474 - val_loss: 16.5369
Epoch 293/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.4376 - val_loss: 14.1389
Epoch 294/1000
1052/1052 [==============================] - 0s 85us/step - loss: 10.9698 - val_loss: 14.4211
Epoch 295/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.0144 - val_loss: 14.5598
Epoch 296/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.1676 - val_loss: 14.8401
Epoch 297/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.2359 - val_loss: 14.1846
Epoch 298/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.2230 - val_loss: 12.6232
Epoch 299/1000
1052/1052 [========

...

Epoch 744/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.7167 - val_loss: 11.6371
Epoch 745/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.5725 - val_loss: 11.2332
Epoch 746/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.8059 - val_loss: 11.6371
Epoch 747/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.2037 - val_loss: 11.6800
Epoch 748/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.1275 - val_loss: 10.9508
Epoch 749/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.7176 - val_loss: 11.4566
Epoch 750/1000
1052/1052 [==============================] - 0s 95us/step - loss: 9.8206 - val_loss: 11.3183
Epoch 751/1000
1052/1052 [==============================] - 0s 95us/step - loss: 9.1770 - val_loss: 11.8449
Epoch 752/1000
1052/1052 [==============================] - 0s 97us/step - loss: 9.5714 - val_loss: 10.7954
Epoch 753/1000
1052/1052 [==

1052/1052 [==============================] - 0s 100us/step - loss: 9.0861 - val_loss: 13.5373
Epoch 820/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.2942 - val_loss: 10.4178
Epoch 821/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.9259 - val_loss: 11.7992
Epoch 822/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.6771 - val_loss: 10.6622
Epoch 823/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.8895 - val_loss: 12.0428
Epoch 824/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.3116 - val_loss: 12.8613
Epoch 825/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.3455 - val_loss: 13.6812
Epoch 826/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.2305 - val_loss: 11.4426
Epoch 827/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.9148 - val_loss: 10.4478
Epoch 828/1000
1052/1052 [================

Epoch 895/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.8485 - val_loss: 10.3481
Epoch 896/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.0351 - val_loss: 10.4031
Epoch 897/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.6806 - val_loss: 11.7927
Epoch 898/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.2338 - val_loss: 11.4087
Epoch 899/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.1360 - val_loss: 10.7722
Epoch 900/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.7734 - val_loss: 11.7990
Epoch 901/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.0804 - val_loss: 10.5640
Epoch 902/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.9323 - val_loss: 10.3132
Epoch 903/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.6921 - val_loss: 10.7599
Epoch 904/1000
1052/1052 [==

Epoch 971/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.6429 - val_loss: 10.6211
Epoch 972/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.0473 - val_loss: 10.8128
Epoch 973/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.9732 - val_loss: 10.0884
Epoch 974/1000
1052/1052 [==============================] - 0s 99us/step - loss: 8.9096 - val_loss: 10.7253
Epoch 975/1000
1052/1052 [==============================] - 0s 97us/step - loss: 9.0274 - val_loss: 10.4229
Epoch 976/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.8149 - val_loss: 10.2351
Epoch 977/1000
1052/1052 [==============================] - 0s 97us/step - loss: 9.0869 - val_loss: 12.5964
Epoch 978/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.7135 - val_loss: 10.8873
Epoch 979/1000
1052/1052 [==============================] - 0s 98us/step - loss: 9.1262 - val_loss: 11.9191
Epoch 980/1000
1052/1052 [==

[array([[ 3.7658505 , -3.653033  ,  1.0955843 ,  0.21308737, -1.6552277 ],
        [-0.34051138,  0.1005788 ,  0.12058192,  0.25269836,  0.36695352],
        [ 0.52621067,  0.09195517,  0.36926234,  0.38295838,  0.2762035 ],
        [-0.1478223 ,  0.06920976, -0.1331796 , -0.09340351,  0.07526465],
        [ 0.4583969 , -2.2980115 ,  1.1225587 ,  0.20043613, -0.1881925 ]],
       dtype=float32),
 array([ 4.666794  , -5.128334  ,  3.055091  ,  0.19168009, -1.2460308 ],
       dtype=float32),
 array([[ 2.1718817 ,  2.323834  ,  1.925013  , -1.8858831 ,  1.7713038 ,
         -1.8209128 , -1.9198476 ,  2.3160708 , -2.2104387 , -2.3922737 ],
        [-1.9812361 , -1.0783396 , -1.5463561 ,  1.9631652 , -1.1482025 ,
          2.094423  ,  1.563533  , -1.6637026 ,  2.0733185 ,  1.649086  ],
        [ 1.2483221 ,  1.6556984 ,  1.7763183 , -1.1809382 ,  1.9065167 ,
         -1.1999451 , -1.1565288 ,  1.4378055 , -1.4539671 , -1.5442106 ],
        [-1.5300195 , -0.64329743, -1.3546942 ,  0.951124

In [25]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_linear_3rd.h5')

In [26]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_relu(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 198us/step - l

1052/1052 [==============================] - 0s 92us/step - loss: 21.3233 - val_loss: 27.3924
Epoch 68/1000
1052/1052 [==============================] - 0s 88us/step - loss: 21.4532 - val_loss: 28.1551
Epoch 69/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.2847 - val_loss: 27.9488
Epoch 70/1000
1052/1052 [==============================] - 0s 91us/step - loss: 22.0314 - val_loss: 27.6982
Epoch 71/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.4642 - val_loss: 28.5785
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 20.9903 - val_loss: 26.6229
Epoch 73/1000
1052/1052 [==============================] - 0s 89us/step - loss: 21.1906 - val_loss: 26.7869
Epoch 74/1000
1052/1052 [==============================] - 0s 91us/step - loss: 20.8544 - val_loss: 26.8088
Epoch 75/1000
1052/1052 [==============================] - 0s 91us/step - loss: 21.1349 - val_loss: 27.9432
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 93us/step - loss: 20.9305 - val_loss: 26.5592
Epoch 143/1000
1052/1052 [==============================] - 0s 87us/step - loss: 20.3624 - val_loss: 25.8656
Epoch 144/1000
1052/1052 [==============================] - 0s 87us/step - loss: 20.7616 - val_loss: 24.9617
Epoch 145/1000
1052/1052 [==============================] - 0s 88us/step - loss: 20.9493 - val_loss: 24.5157
Epoch 146/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.2598 - val_loss: 24.4363
Epoch 147/1000
1052/1052 [==============================] - 0s 89us/step - loss: 19.9917 - val_loss: 25.4127
Epoch 148/1000
1052/1052 [==============================] - 0s 87us/step - loss: 20.3674 - val_loss: 25.0395
Epoch 149/1000
1052/1052 [==============================] - 0s 91us/step - loss: 20.6122 - val_loss: 24.3634
Epoch 150/1000
1052/1052 [==============================] - 0s 93us/step - loss: 21.7996 - val_loss: 26.5654
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 19.3760 - val_loss: 23.6412
Epoch 218/1000
1052/1052 [==============================] - 0s 91us/step - loss: 20.2428 - val_loss: 25.5657
Epoch 219/1000
1052/1052 [==============================] - 0s 90us/step - loss: 20.1090 - val_loss: 24.9479
Epoch 220/1000
1052/1052 [==============================] - 0s 90us/step - loss: 20.2742 - val_loss: 23.6045
Epoch 221/1000
1052/1052 [==============================] - 0s 93us/step - loss: 20.2167 - val_loss: 25.6844
Epoch 222/1000
1052/1052 [==============================] - 0s 93us/step - loss: 21.3071 - val_loss: 24.3206
Epoch 223/1000
1052/1052 [==============================] - 0s 89us/step - loss: 19.6170 - val_loss: 23.9917
Epoch 224/1000
1052/1052 [==============================] - 0s 91us/step - loss: 19.2484 - val_loss: 24.7423
Epoch 225/1000
1052/1052 [==============================] - 0s 93us/step - loss: 19.2441 - val_loss: 24.0408
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 94us/step - loss: 20.3016 - val_loss: 22.2547
Epoch 293/1000
1052/1052 [==============================] - 0s 91us/step - loss: 18.4007 - val_loss: 24.6968
Epoch 294/1000
1052/1052 [==============================] - 0s 95us/step - loss: 19.0920 - val_loss: 23.2550
Epoch 295/1000
1052/1052 [==============================] - 0s 101us/step - loss: 19.3829 - val_loss: 24.0341
Epoch 296/1000
1052/1052 [==============================] - 0s 98us/step - loss: 19.9739 - val_loss: 23.1956
Epoch 297/1000
1052/1052 [==============================] - 0s 100us/step - loss: 19.0521 - val_loss: 22.2500
Epoch 298/1000
1052/1052 [==============================] - 0s 105us/step - loss: 19.1792 - val_loss: 23.8279
Epoch 299/1000
1052/1052 [==============================] - 0s 94us/step - loss: 18.7826 - val_loss: 22.2801
Epoch 300/1000
1052/1052 [==============================] - 0s 114us/step - loss: 18.6464 - val_loss: 22.1205
Epoch 301/1000
1052/1052 [====

...

1052/1052 [==============================] - 0s 91us/step - loss: 17.3908 - val_loss: 22.0229
Epoch 743/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.2586 - val_loss: 22.7190
Epoch 744/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.7996 - val_loss: 23.7539
Epoch 745/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.6067 - val_loss: 22.2089
Epoch 746/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.8756 - val_loss: 23.1378
Epoch 747/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.4172 - val_loss: 23.0251
Epoch 748/1000
1052/1052 [==============================] - 0s 93us/step - loss: 17.2823 - val_loss: 23.7127
Epoch 749/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.3236 - val_loss: 23.0406
Epoch 750/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.3015 - val_loss: 23.1944
Epoch 751/1000
1052/1052 [========

1052/1052 [==============================] - 0s 93us/step - loss: 18.1480 - val_loss: 23.5726
Epoch 818/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.0048 - val_loss: 24.5123
Epoch 819/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.1854 - val_loss: 22.5178
Epoch 820/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.8299 - val_loss: 23.5107
Epoch 821/1000
1052/1052 [==============================] - 0s 92us/step - loss: 18.8215 - val_loss: 23.4184
Epoch 822/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.6245 - val_loss: 22.6495
Epoch 823/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.2398 - val_loss: 22.6719
Epoch 824/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.2424 - val_loss: 22.7430
Epoch 825/1000
1052/1052 [==============================] - 0s 91us/step - loss: 16.8232 - val_loss: 24.3394
Epoch 826/1000
1052/1052 [========

1052/1052 [==============================] - 0s 92us/step - loss: 17.9810 - val_loss: 23.9197
Epoch 893/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.6888 - val_loss: 22.2884
Epoch 894/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.2314 - val_loss: 23.3550
Epoch 895/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.6184 - val_loss: 23.4083
Epoch 896/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.0777 - val_loss: 22.1379
Epoch 897/1000
1052/1052 [==============================] - 0s 90us/step - loss: 16.8201 - val_loss: 23.0318
Epoch 898/1000
1052/1052 [==============================] - 0s 91us/step - loss: 16.8144 - val_loss: 22.0848
Epoch 899/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.1716 - val_loss: 22.8371
Epoch 900/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.9026 - val_loss: 23.3390
Epoch 901/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 17.9055 - val_loss: 23.2119
Epoch 968/1000
1052/1052 [==============================] - 0s 89us/step - loss: 16.9060 - val_loss: 23.3283
Epoch 969/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.6466 - val_loss: 22.6567
Epoch 970/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.6349 - val_loss: 22.2965
Epoch 971/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.0875 - val_loss: 22.7210
Epoch 972/1000
1052/1052 [==============================] - 0s 93us/step - loss: 17.5703 - val_loss: 23.7153
Epoch 973/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.0206 - val_loss: 22.8930
Epoch 974/1000
1052/1052 [==============================] - 0s 86us/step - loss: 17.4505 - val_loss: 22.6968
Epoch 975/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.4011 - val_loss: 24.3332
Epoch 976/1000
1052/1052 [========

[array([[-1.837782  , -0.5893652 ,  0.44344148, -0.37679902,  0.29762483],
        [-0.02188136,  0.85622156,  0.26500577, -0.8885995 , -0.98142815],
        [ 2.4306722 , -0.8199033 ,  0.86364365, -0.74336743, -0.44113207],
        [ 0.05766043,  0.58668154, -0.99283904, -1.4635586 ,  1.4678351 ],
        [ 0.04846913,  0.06993047, -0.71572864,  0.24526842, -0.06539471]],
       dtype=float32),
 array([1.5074065, 3.8480053, 3.0673833, 2.809164 , 2.859303 ],
       dtype=float32),
 array([[-0.4362043 , -1.0618963 , -1.5817282 , -0.57516956,  0.5314147 ,
          1.1288497 , -0.59765995, -0.9144926 ,  1.1658046 , -0.52084994],
        [-0.23517165, -0.82399905, -1.0328338 , -0.410401  ,  1.2093011 ,
          1.9458966 , -0.77957994, -0.56000936,  2.1545522 ,  1.1526929 ],
        [-0.2604112 , -0.47729313,  1.287791  , -1.0578175 ,  0.1511706 ,
          1.3583767 , -1.0426543 , -0.8223603 ,  0.8810432 ,  0.04247031],
        [-0.56300616, -0.23698232, -0.29415813,  0.06415842,  0.929

In [27]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_relu_3rd.h5')

In [28]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_sigmoid(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 215us/step - l

1052/1052 [==============================] - 0s 93us/step - loss: 36.0329 - val_loss: 34.8714
Epoch 67/1000
1052/1052 [==============================] - 0s 91us/step - loss: 29.5630 - val_loss: 28.3904
Epoch 68/1000
1052/1052 [==============================] - 0s 88us/step - loss: 24.9281 - val_loss: 26.4108
Epoch 69/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.8885 - val_loss: 25.2936
Epoch 70/1000
1052/1052 [==============================] - 0s 93us/step - loss: 21.5330 - val_loss: 24.4490
Epoch 71/1000
1052/1052 [==============================] - 0s 91us/step - loss: 20.4463 - val_loss: 23.3754
Epoch 72/1000
1052/1052 [==============================] - 0s 92us/step - loss: 19.5179 - val_loss: 22.7201
Epoch 73/1000
1052/1052 [==============================] - 0s 91us/step - loss: 19.0314 - val_loss: 22.2764
Epoch 74/1000
1052/1052 [==============================] - 0s 91us/step - loss: 18.2419 - val_loss: 22.0600
Epoch 75/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 91us/step - loss: 13.2092 - val_loss: 18.3544
Epoch 142/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.2524 - val_loss: 18.1419
Epoch 143/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.1658 - val_loss: 18.0260
Epoch 144/1000
1052/1052 [==============================] - 0s 92us/step - loss: 13.0860 - val_loss: 18.0323
Epoch 145/1000
1052/1052 [==============================] - 0s 92us/step - loss: 13.0730 - val_loss: 18.5174
Epoch 146/1000
1052/1052 [==============================] - 0s 98us/step - loss: 13.1443 - val_loss: 18.1567
Epoch 147/1000
1052/1052 [==============================] - 0s 92us/step - loss: 12.9380 - val_loss: 18.1548
Epoch 148/1000
1052/1052 [==============================] - 0s 91us/step - loss: 12.8959 - val_loss: 18.2955
Epoch 149/1000
1052/1052 [==============================] - 0s 93us/step - loss: 12.9713 - val_loss: 18.1182
Epoch 150/1000
1052/1052 [========

1052/1052 [==============================] - 0s 91us/step - loss: 10.7670 - val_loss: 15.0895
Epoch 217/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.8491 - val_loss: 15.0762
Epoch 218/1000
1052/1052 [==============================] - 0s 92us/step - loss: 10.7203 - val_loss: 15.0167
Epoch 219/1000
1052/1052 [==============================] - 0s 90us/step - loss: 10.8392 - val_loss: 14.7567
Epoch 220/1000
1052/1052 [==============================] - 0s 92us/step - loss: 10.5801 - val_loss: 14.6174
Epoch 221/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.6671 - val_loss: 14.9045
Epoch 222/1000
1052/1052 [==============================] - 0s 92us/step - loss: 10.6422 - val_loss: 14.7962
Epoch 223/1000
1052/1052 [==============================] - 0s 93us/step - loss: 10.6634 - val_loss: 15.1205
Epoch 224/1000
1052/1052 [==============================] - 0s 94us/step - loss: 10.5972 - val_loss: 14.4012
Epoch 225/1000
1052/1052 [========

Epoch 292/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2626 - val_loss: 9.8308
Epoch 293/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.3861 - val_loss: 9.8776
Epoch 294/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.3020 - val_loss: 10.0324
Epoch 295/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.2767 - val_loss: 9.6250
Epoch 296/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.1723 - val_loss: 10.0233
Epoch 297/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2162 - val_loss: 10.2133
Epoch 298/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3154 - val_loss: 9.9215
Epoch 299/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.5280 - val_loss: 9.6834
Epoch 300/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.0899 - val_loss: 10.1982
Epoch 301/1000
1052/1052 [=======

...

1052/1052 [==============================] - 0s 92us/step - loss: 4.5456 - val_loss: 5.6610
Epoch 749/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.5288 - val_loss: 5.5613
Epoch 750/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.5186 - val_loss: 5.5830
Epoch 751/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.6060 - val_loss: 5.8317
Epoch 752/1000
1052/1052 [==============================] - 0s 92us/step - loss: 4.5705 - val_loss: 6.4261
Epoch 753/1000
1052/1052 [==============================] - 0s 98us/step - loss: 4.6388 - val_loss: 5.6269
Epoch 754/1000
1052/1052 [==============================] - 0s 90us/step - loss: 4.5520 - val_loss: 5.6447
Epoch 755/1000
1052/1052 [==============================] - 0s 93us/step - loss: 4.5121 - val_loss: 5.6342
Epoch 756/1000
1052/1052 [==============================] - 0s 89us/step - loss: 4.6369 - val_loss: 6.1160
Epoch 757/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 90us/step - loss: 4.5662 - val_loss: 5.4120
Epoch 825/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.5046 - val_loss: 5.8982
Epoch 826/1000
1052/1052 [==============================] - 0s 92us/step - loss: 4.5170 - val_loss: 5.7385
Epoch 827/1000
1052/1052 [==============================] - 0s 90us/step - loss: 4.6398 - val_loss: 5.5143
Epoch 828/1000
1052/1052 [==============================] - 0s 89us/step - loss: 4.5309 - val_loss: 5.5471
Epoch 829/1000
1052/1052 [==============================] - 0s 92us/step - loss: 4.5659 - val_loss: 5.4652
Epoch 830/1000
1052/1052 [==============================] - 0s 97us/step - loss: 4.5349 - val_loss: 6.0062
Epoch 831/1000
1052/1052 [==============================] - 0s 93us/step - loss: 4.4289 - val_loss: 5.5631
Epoch 832/1000
1052/1052 [==============================] - 0s 92us/step - loss: 4.5286 - val_loss: 5.7457
Epoch 833/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 105us/step - loss: 4.5320 - val_loss: 5.7489
Epoch 901/1000
1052/1052 [==============================] - 0s 94us/step - loss: 4.4892 - val_loss: 5.8465
Epoch 902/1000
1052/1052 [==============================] - 0s 93us/step - loss: 4.5982 - val_loss: 5.4372
Epoch 903/1000
1052/1052 [==============================] - 0s 94us/step - loss: 4.6170 - val_loss: 5.5776
Epoch 904/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.6762 - val_loss: 5.7259
Epoch 905/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.5134 - val_loss: 5.9842
Epoch 906/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.7019 - val_loss: 6.1937
Epoch 907/1000
1052/1052 [==============================] - 0s 93us/step - loss: 4.5305 - val_loss: 5.3993
Epoch 908/1000
1052/1052 [==============================] - 0s 90us/step - loss: 4.6253 - val_loss: 5.6036
Epoch 909/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 93us/step - loss: 4.3617 - val_loss: 5.6300
Epoch 977/1000
1052/1052 [==============================] - 0s 90us/step - loss: 4.4127 - val_loss: 5.6321
Epoch 978/1000
1052/1052 [==============================] - 0s 96us/step - loss: 4.4269 - val_loss: 5.5227
Epoch 979/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.3358 - val_loss: 5.8180
Epoch 980/1000
1052/1052 [==============================] - 0s 92us/step - loss: 4.4641 - val_loss: 5.4700
Epoch 981/1000
1052/1052 [==============================] - 0s 89us/step - loss: 4.4810 - val_loss: 5.6807
Epoch 982/1000
1052/1052 [==============================] - 0s 91us/step - loss: 4.5506 - val_loss: 5.7896
Epoch 983/1000
1052/1052 [==============================] - 0s 94us/step - loss: 4.3482 - val_loss: 5.8133
Epoch 984/1000
1052/1052 [==============================] - 0s 93us/step - loss: 4.4730 - val_loss: 5.6005
Epoch 985/1000
1052/1052 [==========================

[array([[-6.002497  , -0.00703184, -5.7662573 , -0.6565843 , -1.1423743 ],
        [ 0.34317157,  2.3304763 , -2.1094139 , -2.597349  , -1.2172843 ],
        [-0.15042885, -0.6251251 , -0.5988963 , -3.4287236 , -0.79466593],
        [ 0.13265225, -0.14904036,  0.32933375,  0.28756794,  0.46526277],
        [-0.7206939 , -3.084812  , -0.7857606 ,  2.2460759 , -0.27039552]],
       dtype=float32),
 array([-2.7541926,  0.4257172, -1.3515099, -1.1421643,  2.4124908],
       dtype=float32),
 array([[  3.9182904 ,  -1.5499545 ,   1.0811263 ,   8.676617  ,
          -2.0654542 ,   4.599918  ,  -5.2806234 ,  14.724277  ,
          -2.0936975 ,   2.4185119 ],
        [  8.563808  ,   1.461225  ,   0.6538052 ,  16.72399   ,
          -0.4404111 ,   6.5120573 , -16.366434  ,   1.4232162 ,
          -3.1343946 ,   0.1874299 ],
        [ -0.06170878,   3.6289163 ,  -0.02916546,  -1.9480157 ,
          -0.8044095 ,  25.436583  ,  -4.764775  ,  12.029613  ,
           1.9831724 ,  -1.5668567 ],
     

In [29]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_sigmoid_3rd.h5')

In [30]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_tanh(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 227us/step - l

1052/1052 [==============================] - 0s 93us/step - loss: 47.3749 - val_loss: 43.9156
Epoch 67/1000
1052/1052 [==============================] - 0s 94us/step - loss: 47.3737 - val_loss: 43.9105
Epoch 68/1000
1052/1052 [==============================] - 0s 90us/step - loss: 47.3790 - val_loss: 43.9188
Epoch 69/1000
1052/1052 [==============================] - 0s 95us/step - loss: 47.3836 - val_loss: 43.9171
Epoch 70/1000
1052/1052 [==============================] - 0s 89us/step - loss: 47.3762 - val_loss: 43.9299
Epoch 71/1000
1052/1052 [==============================] - 0s 91us/step - loss: 47.3808 - val_loss: 43.9200
Epoch 72/1000
1052/1052 [==============================] - 0s 92us/step - loss: 47.3866 - val_loss: 43.9393
Epoch 73/1000
1052/1052 [==============================] - 0s 89us/step - loss: 47.3715 - val_loss: 43.9200
Epoch 74/1000
1052/1052 [==============================] - 0s 91us/step - loss: 47.3765 - val_loss: 43.9059
Epoch 75/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 91us/step - loss: 12.9178 - val_loss: 16.3838
Epoch 142/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.6195 - val_loss: 15.5972
Epoch 143/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.3979 - val_loss: 15.7245
Epoch 144/1000
1052/1052 [==============================] - 0s 94us/step - loss: 12.2502 - val_loss: 15.4384
Epoch 145/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.9812 - val_loss: 15.1637
Epoch 146/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.8767 - val_loss: 14.5179
Epoch 147/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.6684 - val_loss: 15.2851
Epoch 148/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.2533 - val_loss: 14.6251
Epoch 149/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.3374 - val_loss: 14.4192
Epoch 150/1000
1052/1052 [========

1052/1052 [==============================] - 0s 93us/step - loss: 6.9753 - val_loss: 8.7996
Epoch 218/1000
1052/1052 [==============================] - 0s 91us/step - loss: 6.7868 - val_loss: 7.9373
Epoch 219/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.7498 - val_loss: 7.8455
Epoch 220/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.6543 - val_loss: 8.1256
Epoch 221/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.6938 - val_loss: 7.4608
Epoch 222/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.4814 - val_loss: 7.6047
Epoch 223/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.4579 - val_loss: 8.1878
Epoch 224/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.5471 - val_loss: 7.2433
Epoch 225/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.3518 - val_loss: 7.2853
Epoch 226/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 89us/step - loss: 6.3071 - val_loss: 7.5731
Epoch 294/1000
1052/1052 [==============================] - 0s 103us/step - loss: 6.0994 - val_loss: 7.2196
Epoch 295/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.9811 - val_loss: 6.9761
Epoch 296/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.1358 - val_loss: 7.1787
Epoch 297/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.0053 - val_loss: 7.1041
Epoch 298/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.0246 - val_loss: 7.0382
Epoch 299/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.9703 - val_loss: 6.9716
Epoch 300/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.9872 - val_loss: 7.1542
Epoch 301/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.9583 - val_loss: 6.8149
Epoch 302/1000
1052/1052 [=========================

...

1052/1052 [==============================] - 0s 94us/step - loss: 5.4186 - val_loss: 6.3215
Epoch 750/1000
1052/1052 [==============================] - 0s 97us/step - loss: 5.4971 - val_loss: 6.8031
Epoch 751/1000
1052/1052 [==============================] - 0s 99us/step - loss: 5.4490 - val_loss: 6.4458
Epoch 752/1000
1052/1052 [==============================] - 0s 95us/step - loss: 5.5262 - val_loss: 6.0860
Epoch 753/1000
1052/1052 [==============================] - 0s 97us/step - loss: 5.4568 - val_loss: 6.4106
Epoch 754/1000
1052/1052 [==============================] - 0s 96us/step - loss: 5.4489 - val_loss: 6.5568
Epoch 755/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.6428 - val_loss: 6.1700
Epoch 756/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.5853 - val_loss: 6.7286
Epoch 757/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.6464 - val_loss: 6.5254
Epoch 758/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 90us/step - loss: 5.3735 - val_loss: 6.9138
Epoch 826/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.4879 - val_loss: 6.5980
Epoch 827/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.4160 - val_loss: 6.6892
Epoch 828/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.3562 - val_loss: 6.5224
Epoch 829/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.5862 - val_loss: 6.9998
Epoch 830/1000
1052/1052 [==============================] - 0s 94us/step - loss: 5.3496 - val_loss: 6.5456
Epoch 831/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.3625 - val_loss: 6.5976
Epoch 832/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.4020 - val_loss: 6.5086
Epoch 833/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.4555 - val_loss: 6.9584
Epoch 834/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 95us/step - loss: 5.4511 - val_loss: 6.4420
Epoch 902/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.5755 - val_loss: 6.2757
Epoch 903/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.3669 - val_loss: 6.5357
Epoch 904/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.4049 - val_loss: 6.6110
Epoch 905/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.3673 - val_loss: 6.4628
Epoch 906/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.4402 - val_loss: 6.9946
Epoch 907/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.4036 - val_loss: 6.5966
Epoch 908/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.4616 - val_loss: 7.0414
Epoch 909/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.4252 - val_loss: 6.3416
Epoch 910/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 91us/step - loss: 5.4919 - val_loss: 6.5405
Epoch 978/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.4240 - val_loss: 6.3454
Epoch 979/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.5900 - val_loss: 6.2855
Epoch 980/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.4388 - val_loss: 6.6897
Epoch 981/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.5275 - val_loss: 6.6639
Epoch 982/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.2716 - val_loss: 6.0532
Epoch 983/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.4034 - val_loss: 6.5081
Epoch 984/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.3031 - val_loss: 6.3575
Epoch 985/1000
1052/1052 [==============================] - 0s 95us/step - loss: 5.3707 - val_loss: 6.5118
Epoch 986/1000
1052/1052 [==========================

[array([[-5.0920196 , -0.26277733, -1.1332595 , -1.9088807 , -0.21591893],
        [-0.17232013, -0.3951683 , -1.8251202 , -0.07073447,  0.15259446],
        [-0.42864627, -3.0473502 , -0.6090515 , -0.97683996,  0.11518971],
        [ 0.10663496,  0.06479504,  0.39682177,  0.50392634,  0.19545107],
        [-0.6286738 , -0.4108839 , -0.06146777, -0.04667597, -0.12458198]],
       dtype=float32),
 array([-1.7349565 , -1.2558692 ,  1.5949044 ,  0.18641746, -0.6460914 ],
       dtype=float32),
 array([[ -2.0011518 ,  -2.2144256 ,   0.27939513,  17.805655  ,
          -1.0098608 ,  -1.201983  ,  -0.9318017 ,  -0.61543185,
          17.100153  ,  23.035322  ],
        [  0.36211967,   0.4207473 , -36.443237  ,   5.472167  ,
          -2.799814  ,  -0.42285696,  -0.5090485 ,  -1.9035214 ,
           5.5702467 ,   0.46516478],
        [  1.7966375 ,   0.6266629 ,   0.48500353,  17.445292  ,
          -0.346704  ,  -0.05677722,  -0.44733915,  -0.41755545,
          27.375526  ,   0.4886184 ],


In [31]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_tanh_3rd.h5')

In [32]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_linear(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 664us/step - loss:

Epoch 69/1000
274/274 [==============================] - 0s 98us/step - loss: 7.5042 - val_loss: 9.0066
Epoch 70/1000
274/274 [==============================] - ETA: 0s - loss: 7.845 - 0s 98us/step - loss: 7.6119 - val_loss: 9.0384
Epoch 71/1000
274/274 [==============================] - 0s 95us/step - loss: 7.1910 - val_loss: 8.8557
Epoch 72/1000
274/274 [==============================] - 0s 95us/step - loss: 7.1903 - val_loss: 8.8043
Epoch 73/1000
274/274 [==============================] - 0s 95us/step - loss: 7.2424 - val_loss: 8.4116
Epoch 74/1000
274/274 [==============================] - 0s 93us/step - loss: 7.2392 - val_loss: 8.6803
Epoch 75/1000
274/274 [==============================] - 0s 95us/step - loss: 7.2011 - val_loss: 8.7476
Epoch 76/1000
274/274 [==============================] - 0s 95us/step - loss: 7.1438 - val_loss: 8.4847
Epoch 77/1000
274/274 [==============================] - 0s 95us/step - loss: 7.8014 - val_loss: 8.6297
Epoch 78/1000
274/274 [=================

274/274 [==============================] - 0s 95us/step - loss: 6.8025 - val_loss: 8.0851
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8163 - val_loss: 8.3585
Epoch 148/1000
274/274 [==============================] - 0s 96us/step - loss: 6.7120 - val_loss: 8.4917
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 6.7521 - val_loss: 8.2397
Epoch 150/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8413 - val_loss: 8.3727
Epoch 151/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0741 - val_loss: 8.6625
Epoch 152/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8450 - val_loss: 8.4007
Epoch 153/1000
274/274 [==============================] - 0s 94us/step - loss: 7.0415 - val_loss: 8.5407
Epoch 154/1000
274/274 [==============================] - 0s 102us/step - loss: 7.0575 - val_loss: 8.5966
Epoch 155/1000
274/274 [==============================] - 0s 102us

274/274 [==============================] - 0s 102us/step - loss: 6.5984 - val_loss: 8.0767
Epoch 224/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7135 - val_loss: 8.0592
Epoch 225/1000
274/274 [==============================] - 0s 95us/step - loss: 6.9545 - val_loss: 8.0884
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8105 - val_loss: 8.2885
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6090 - val_loss: 8.2467
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6311 - val_loss: 8.2186
Epoch 229/1000
274/274 [==============================] - 0s 105us/step - loss: 6.6094 - val_loss: 8.1707
Epoch 230/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6099 - val_loss: 8.1463
Epoch 231/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6228 - val_loss: 8.2584
Epoch 232/1000
274/274 [==============================] - 0s 106us

274/274 [==============================] - 0s 102us/step - loss: 6.6836 - val_loss: 7.9795
Epoch 301/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8597 - val_loss: 7.8270
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5403 - val_loss: 7.9392
Epoch 303/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5282 - val_loss: 8.0692
Epoch 304/1000
274/274 [==============================] - 0s 99us/step - loss: 6.6412 - val_loss: 8.0824
Epoch 305/1000
274/274 [==============================] - 0s 107us/step - loss: 6.6678 - val_loss: 8.2119
Epoch 306/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7346 - val_loss: 8.4016
Epoch 307/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7696 - val_loss: 7.9409
Epoch 308/1000
274/274 [==============================] - 0s 106us/step - loss: 6.7729 - val_loss: 7.9464
Epoch 309/1000
274/274 [==============================] - 0s 105us

...

274/274 [==============================] - 0s 98us/step - loss: 6.5074 - val_loss: 8.0961
Epoch 764/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5497 - val_loss: 8.1656
Epoch 765/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8764 - val_loss: 7.8783
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6997 - val_loss: 7.9969
Epoch 767/1000
274/274 [==============================] - 0s 94us/step - loss: 6.6166 - val_loss: 8.2006
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4852 - val_loss: 8.1544
Epoch 769/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5568 - val_loss: 8.0087
Epoch 770/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4253 - val_loss: 8.1846
Epoch 771/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5325 - val_loss: 8.1025
Epoch 772/1000
274/274 [==============================] - 0s 98us/st

274/274 [==============================] - 0s 95us/step - loss: 6.3778 - val_loss: 8.2995
Epoch 841/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6241 - val_loss: 8.5091
Epoch 842/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4607 - val_loss: 8.3010
Epoch 843/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7108 - val_loss: 8.2969
Epoch 844/1000
274/274 [==============================] - 0s 94us/step - loss: 6.6446 - val_loss: 8.2799
Epoch 845/1000
274/274 [==============================] - 0s 98us/step - loss: 6.8570 - val_loss: 8.5198
Epoch 846/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6066 - val_loss: 8.3271
Epoch 847/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6290 - val_loss: 8.2453
Epoch 848/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4943 - val_loss: 8.3247
Epoch 849/1000
274/274 [==============================] - 0s 102us/

274/274 [==============================] - 0s 98us/step - loss: 6.3954 - val_loss: 8.3374
Epoch 918/1000
274/274 [==============================] - 0s 106us/step - loss: 6.3927 - val_loss: 8.4145
Epoch 919/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3501 - val_loss: 8.3103
Epoch 920/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4328 - val_loss: 8.2900
Epoch 921/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3731 - val_loss: 8.1240
Epoch 922/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4896 - val_loss: 8.2262
Epoch 923/1000
274/274 [==============================] - 0s 106us/step - loss: 6.7082 - val_loss: 8.3961
Epoch 924/1000
274/274 [==============================] - 0s 104us/step - loss: 6.5002 - val_loss: 8.1301
Epoch 925/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4267 - val_loss: 8.1068
Epoch 926/1000
274/274 [==============================] - 0s 98u

274/274 [==============================] - 0s 102us/step - loss: 6.4272 - val_loss: 8.3854
Epoch 995/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5074 - val_loss: 8.1805
Epoch 996/1000
274/274 [==============================] - 0s 101us/step - loss: 6.3228 - val_loss: 8.1996
Epoch 997/1000
274/274 [==============================] - 0s 97us/step - loss: 6.5378 - val_loss: 8.3418
Epoch 998/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4074 - val_loss: 8.4241
Epoch 999/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4945 - val_loss: 8.3948
Epoch 1000/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4947 - val_loss: 8.1553
7.9775951191530385


[array([[-1.4557352 , -1.4603144 ,  0.00431802,  0.77965033,  2.313337  ],
        [-3.5607557 ,  1.6285157 ,  0.39313743, -0.7056128 ,  2.1320572 ],
        [-1.1097881 , -1.0705929 ,  0.33621332,  0.47667104, -1.7753912 ],
        [ 1.692369  , -1.8316298 ,  0.32906792,  0.74673444,  0.02705619],
        [ 0.36476725,  1.1165972 ,  0.17069489, -0.68670243, -0.1489947 ],
        [-1.0247089 ,  1.1996968 , -0.12543444, -0.32168117,  2.2006674 ],
        [ 0.10253755, -0.09364851, -0.39647064,  0.4227315 , -0.32811204]],
       dtype=float32),
 array([ 2.154013  ,  0.14802763,  0.14071143, -0.28499466,  2.2767975 ],
       dtype=float32),
 array([[-0.51445085, -0.7866926 , -0.04045059, -0.36563706,  0.5629598 ,
         -0.56360126,  0.05057036, -0.83417493, -0.7739682 ,  0.3012023 ],
        [-0.20488733, -0.36186048, -0.0580551 , -0.561227  ,  0.77993053,
         -0.44420603,  0.85518086,  0.2302954 , -0.39225513,  0.07703407],
        [-1.0661572 , -0.9134894 , -1.2018791 , -1.42376

In [33]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_linear_3rd.h5')

In [34]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_relu(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 764us/step - loss:

274/274 [==============================] - 0s 106us/step - loss: 7.4420 - val_loss: 7.7083
Epoch 70/1000
274/274 [==============================] - 0s 98us/step - loss: 7.3221 - val_loss: 7.6398
Epoch 71/1000
274/274 [==============================] - 0s 95us/step - loss: 7.3279 - val_loss: 7.8964
Epoch 72/1000
274/274 [==============================] - 0s 102us/step - loss: 7.3601 - val_loss: 7.9845
Epoch 73/1000
274/274 [==============================] - 0s 91us/step - loss: 7.2657 - val_loss: 7.7490
Epoch 74/1000
274/274 [==============================] - 0s 94us/step - loss: 7.3529 - val_loss: 7.4920
Epoch 75/1000
274/274 [==============================] - 0s 91us/step - loss: 7.3085 - val_loss: 7.6299
Epoch 76/1000
274/274 [==============================] - 0s 146us/step - loss: 7.2583 - val_loss: 7.6100
Epoch 77/1000
274/274 [==============================] - 0s 131us/step - loss: 7.2099 - val_loss: 8.0254
Epoch 78/1000
274/274 [==============================] - 0s 116us/step - l

Epoch 147/1000
274/274 [==============================] - 0s 98us/step - loss: 6.9877 - val_loss: 7.1129
Epoch 148/1000
274/274 [==============================] - 0s 106us/step - loss: 7.1166 - val_loss: 6.9061
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9084 - val_loss: 7.3079
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9328 - val_loss: 7.0758
Epoch 151/1000
274/274 [==============================] - 0s 106us/step - loss: 6.9478 - val_loss: 7.0503
Epoch 152/1000
274/274 [==============================] - 0s 102us/step - loss: 7.0050 - val_loss: 7.2338
Epoch 153/1000
274/274 [==============================] - 0s 104us/step - loss: 7.0779 - val_loss: 6.9788
Epoch 154/1000
274/274 [==============================] - 0s 109us/step - loss: 7.0040 - val_loss: 6.9363
Epoch 155/1000
274/274 [==============================] - 0s 109us/step - loss: 6.9748 - val_loss: 7.0568
Epoch 156/1000
274/274 [=======================

274/274 [==============================] - 0s 102us/step - loss: 6.7592 - val_loss: 6.8328
Epoch 225/1000
274/274 [==============================] - 0s 95us/step - loss: 7.0220 - val_loss: 7.1724
Epoch 226/1000
274/274 [==============================] - 0s 91us/step - loss: 6.9099 - val_loss: 6.9841
Epoch 227/1000
274/274 [==============================] - 0s 127us/step - loss: 6.9732 - val_loss: 7.2916
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7861 - val_loss: 6.8519
Epoch 229/1000
274/274 [==============================] - 0s 95us/step - loss: 7.3025 - val_loss: 7.7658
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9002 - val_loss: 6.9565
Epoch 231/1000
274/274 [==============================] - 0s 113us/step - loss: 6.8007 - val_loss: 7.1125
Epoch 232/1000
274/274 [==============================] - 0s 95us/step - loss: 6.9379 - val_loss: 6.7678
Epoch 233/1000
274/274 [==============================] - 0s 106us

274/274 [==============================] - 0s 116us/step - loss: 7.1087 - val_loss: 7.3064
Epoch 302/1000
274/274 [==============================] - 0s 113us/step - loss: 6.9491 - val_loss: 7.0295
Epoch 303/1000
274/274 [==============================] - 0s 109us/step - loss: 6.7563 - val_loss: 7.1663
Epoch 304/1000
274/274 [==============================] - 0s 109us/step - loss: 6.8868 - val_loss: 6.9224
Epoch 305/1000
274/274 [==============================] - 0s 109us/step - loss: 6.8419 - val_loss: 7.0451
Epoch 306/1000
274/274 [==============================] - 0s 106us/step - loss: 6.8663 - val_loss: 7.0575
Epoch 307/1000
274/274 [==============================] - 0s 124us/step - loss: 6.7774 - val_loss: 6.9158
Epoch 308/1000
274/274 [==============================] - 0s 127us/step - loss: 6.8180 - val_loss: 7.1030
Epoch 309/1000
274/274 [==============================] - 0s 120us/step - loss: 6.6731 - val_loss: 6.8995
Epoch 310/1000
274/274 [==============================] - 0s 

...

274/274 [==============================] - 0s 95us/step - loss: 5.4752 - val_loss: 8.5213
Epoch 766/1000
274/274 [==============================] - 0s 124us/step - loss: 5.5126 - val_loss: 9.1839
Epoch 767/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1609 - val_loss: 8.9026
Epoch 768/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0919 - val_loss: 8.5145
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2727 - val_loss: 8.8279
Epoch 770/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4141 - val_loss: 8.6990
Epoch 771/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1502 - val_loss: 8.9124
Epoch 772/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1004 - val_loss: 9.0552
Epoch 773/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1299 - val_loss: 8.9982
Epoch 774/1000
274/274 [==============================] - 0s 95us

274/274 [==============================] - 0s 105us/step - loss: 5.1004 - val_loss: 8.9309
Epoch 843/1000
274/274 [==============================] - 0s 108us/step - loss: 5.1469 - val_loss: 9.1525
Epoch 844/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1865 - val_loss: 8.7211
Epoch 845/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0758 - val_loss: 9.0680
Epoch 846/1000
274/274 [==============================] - 0s 113us/step - loss: 5.1777 - val_loss: 8.8356
Epoch 847/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1962 - val_loss: 9.1194
Epoch 848/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1296 - val_loss: 9.6413
Epoch 849/1000
274/274 [==============================] - 0s 107us/step - loss: 5.2516 - val_loss: 9.0590
Epoch 850/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4286 - val_loss: 9.2629
Epoch 851/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 101us/step - loss: 5.3117 - val_loss: 9.3613
Epoch 920/1000
274/274 [==============================] - 0s 104us/step - loss: 5.1860 - val_loss: 9.5225
Epoch 921/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2825 - val_loss: 8.8352
Epoch 922/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3968 - val_loss: 10.0526
Epoch 923/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3154 - val_loss: 8.6550
Epoch 924/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1914 - val_loss: 8.8256
Epoch 925/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1142 - val_loss: 9.1581
Epoch 926/1000
274/274 [==============================] - 0s 99us/step - loss: 5.0979 - val_loss: 9.0991
Epoch 927/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1017 - val_loss: 9.0938
Epoch 928/1000
274/274 [==============================] - 0s 91us/s

274/274 [==============================] - 0s 96us/step - loss: 5.3428 - val_loss: 8.8496
Epoch 998/1000
274/274 [==============================] - 0s 89us/step - loss: 5.3941 - val_loss: 9.3568
Epoch 999/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2890 - val_loss: 8.8301
Epoch 1000/1000
274/274 [==============================] - 0s 95us/step - loss: 4.9969 - val_loss: 9.0360
6.628869258751304


[array([[-0.61748075,  0.8647203 , -0.43723792,  0.62333024, -0.6412967 ],
        [-0.9120634 , -1.9209827 ,  1.348925  , -0.9756895 ,  0.91940546],
        [-0.2349958 ,  0.2790235 , -0.53637767, -0.90401083, -0.4818502 ],
        [ 0.03034029,  0.26190096, -2.1994724 , -0.554997  ,  0.1632381 ],
        [-1.0890766 ,  1.0015293 , -0.20767866, -0.6880193 ,  0.4337065 ],
        [-0.06539302, -0.54370743,  0.28301778, -0.6741587 ,  1.605055  ],
        [-3.208143  , -0.8670386 ,  0.3909985 ,  1.3989067 , -0.20494041]],
       dtype=float32),
 array([1.0712596 , 1.1262215 , 0.72805077, 0.93181837, 1.7721711 ],
       dtype=float32),
 array([[ 0.19756511,  0.04862389, -0.06738003, -0.17663118,  0.31466728,
         -0.23976696,  1.2300557 ,  1.953325  ,  0.64703614,  1.7743678 ],
        [ 0.83058494, -0.3730811 , -0.547277  ,  0.5496046 ,  1.2318879 ,
         -0.37461796,  0.3495903 ,  1.5899234 ,  0.909044  ,  0.3727565 ],
        [ 0.5971443 , -0.72728467, -0.7020506 , -0.55599535, 

In [35]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_relu_3rd.h5')

In [36]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_sigmoid(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 787us/step - loss:

Epoch 68/1000
274/274 [==============================] - 0s 109us/step - loss: 15.7409 - val_loss: 19.7821
Epoch 69/1000
274/274 [==============================] - 0s 102us/step - loss: 14.6425 - val_loss: 18.4947
Epoch 70/1000
274/274 [==============================] - 0s 105us/step - loss: 13.7031 - val_loss: 17.2793
Epoch 71/1000
274/274 [==============================] - 0s 106us/step - loss: 12.6641 - val_loss: 16.6329
Epoch 72/1000
274/274 [==============================] - 0s 106us/step - loss: 12.1221 - val_loss: 16.5681
Epoch 73/1000
274/274 [==============================] - 0s 111us/step - loss: 11.5458 - val_loss: 15.8215
Epoch 74/1000
274/274 [==============================] - 0s 106us/step - loss: 11.1484 - val_loss: 15.3532
Epoch 75/1000
274/274 [==============================] - 0s 98us/step - loss: 10.6880 - val_loss: 15.1859
Epoch 76/1000
274/274 [==============================] - 0s 97us/step - loss: 10.4265 - val_loss: 14.9478
Epoch 77/1000
274/274 [================

Epoch 145/1000
274/274 [==============================] - 0s 105us/step - loss: 6.4756 - val_loss: 10.2606
Epoch 146/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4600 - val_loss: 10.1262
Epoch 147/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4946 - val_loss: 10.1308
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5409 - val_loss: 9.9476
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5093 - val_loss: 10.2372
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4220 - val_loss: 9.9985
Epoch 151/1000
274/274 [==============================] - 0s 106us/step - loss: 6.5219 - val_loss: 10.0026
Epoch 152/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5347 - val_loss: 10.3416
Epoch 153/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4422 - val_loss: 9.9850
Epoch 154/1000
274/274 [====================

274/274 [==============================] - 0s 106us/step - loss: 6.0387 - val_loss: 9.7625
Epoch 223/1000
274/274 [==============================] - 0s 131us/step - loss: 6.0578 - val_loss: 9.8207
Epoch 224/1000
274/274 [==============================] - 0s 105us/step - loss: 6.0612 - val_loss: 9.7845
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0600 - val_loss: 9.7213
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0708 - val_loss: 9.9655
Epoch 227/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0634 - val_loss: 9.8464
Epoch 228/1000
274/274 [==============================] - 0s 106us/step - loss: 6.1173 - val_loss: 9.8546
Epoch 229/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0766 - val_loss: 10.0598
Epoch 230/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0620 - val_loss: 9.7594
Epoch 231/1000
274/274 [==============================] - 0s 102u

274/274 [==============================] - 0s 102us/step - loss: 5.8035 - val_loss: 9.5351
Epoch 300/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7738 - val_loss: 9.7105
Epoch 301/1000
274/274 [==============================] - 0s 96us/step - loss: 5.7827 - val_loss: 9.3602
Epoch 302/1000
274/274 [==============================] - 0s 101us/step - loss: 5.7565 - val_loss: 9.4823
Epoch 303/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7536 - val_loss: 9.5132
Epoch 304/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7567 - val_loss: 9.5543
Epoch 305/1000
274/274 [==============================] - 0s 105us/step - loss: 5.7213 - val_loss: 9.4824
Epoch 306/1000
274/274 [==============================] - 0s 109us/step - loss: 5.7444 - val_loss: 9.4584
Epoch 307/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7433 - val_loss: 9.5391
Epoch 308/1000
274/274 [==============================] - 0s 100

...

274/274 [==============================] - 0s 98us/step - loss: 4.8358 - val_loss: 8.9424
Epoch 761/1000
274/274 [==============================] - 0s 110us/step - loss: 4.8824 - val_loss: 8.7735
Epoch 762/1000
274/274 [==============================] - 0s 102us/step - loss: 4.8435 - val_loss: 8.7691
Epoch 763/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8408 - val_loss: 8.9044
Epoch 764/1000
274/274 [==============================] - 0s 107us/step - loss: 4.8156 - val_loss: 8.7285
Epoch 765/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8727 - val_loss: 8.7674
Epoch 766/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8053 - val_loss: 8.9964
Epoch 767/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8813 - val_loss: 8.8173
Epoch 768/1000
274/274 [==============================] - 0s 106us/step - loss: 4.8647 - val_loss: 8.8138
Epoch 769/1000
274/274 [==============================] - 0s 102us

274/274 [==============================] - 0s 102us/step - loss: 4.7285 - val_loss: 9.0921
Epoch 838/1000
274/274 [==============================] - 0s 102us/step - loss: 4.7124 - val_loss: 9.1558
Epoch 839/1000
274/274 [==============================] - 0s 107us/step - loss: 4.7109 - val_loss: 9.2083
Epoch 840/1000
274/274 [==============================] - 0s 94us/step - loss: 4.7404 - val_loss: 9.1573
Epoch 841/1000
274/274 [==============================] - 0s 102us/step - loss: 4.7990 - val_loss: 9.1674
Epoch 842/1000
274/274 [==============================] - 0s 100us/step - loss: 4.6889 - val_loss: 9.3021
Epoch 843/1000
274/274 [==============================] - 0s 106us/step - loss: 4.7616 - val_loss: 8.9596
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7186 - val_loss: 9.1363
Epoch 845/1000
274/274 [==============================] - 0s 102us/step - loss: 4.7302 - val_loss: 9.4410
Epoch 846/1000
274/274 [==============================] - 0s 10

274/274 [==============================] - 0s 102us/step - loss: 4.6125 - val_loss: 9.3836
Epoch 915/1000
274/274 [==============================] - 0s 96us/step - loss: 4.6282 - val_loss: 9.4326
Epoch 916/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5955 - val_loss: 9.3202
Epoch 917/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5782 - val_loss: 9.4691
Epoch 918/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6901 - val_loss: 9.3871
Epoch 919/1000
274/274 [==============================] - 0s 98us/step - loss: 4.6057 - val_loss: 9.6202
Epoch 920/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6088 - val_loss: 9.4425
Epoch 921/1000
274/274 [==============================] - 0s 95us/step - loss: 4.6805 - val_loss: 9.5094
Epoch 922/1000
274/274 [==============================] - 0s 106us/step - loss: 4.5166 - val_loss: 9.2443
Epoch 923/1000
274/274 [==============================] - 0s 98us

274/274 [==============================] - 0s 106us/step - loss: 4.4760 - val_loss: 9.6569
Epoch 992/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5329 - val_loss: 9.4956
Epoch 993/1000
274/274 [==============================] - 0s 99us/step - loss: 4.5648 - val_loss: 9.3482
Epoch 994/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5117 - val_loss: 9.7939
Epoch 995/1000
274/274 [==============================] - 0s 101us/step - loss: 4.5574 - val_loss: 9.6715
Epoch 996/1000
274/274 [==============================] - 0s 97us/step - loss: 4.5010 - val_loss: 9.5132
Epoch 997/1000
274/274 [==============================] - 0s 102us/step - loss: 4.4867 - val_loss: 9.6195
Epoch 998/1000
274/274 [==============================] - 0s 106us/step - loss: 4.5692 - val_loss: 9.6701
Epoch 999/1000
274/274 [==============================] - 0s 106us/step - loss: 4.5507 - val_loss: 9.3355
Epoch 1000/1000
274/274 [==============================] - 0s 10

[array([[ 1.0330298 , -0.36562768,  0.68187046,  2.1354477 , -1.0244323 ],
        [-2.025833  ,  0.34108075, -1.1590474 ,  2.5963187 ,  2.9006069 ],
        [-0.37135696, -0.3219384 ,  1.7971343 ,  2.5317461 ,  0.23362406],
        [-3.5259588 ,  0.36422715,  2.7488182 ,  1.8292645 , -2.7725656 ],
        [ 0.78541327, -0.06564779,  1.6035751 ,  3.2468057 ,  1.183425  ],
        [ 0.82975286,  0.27237663,  0.4746627 , -1.6341947 ,  3.24872   ],
        [-0.23688275,  0.07456251, -4.6085687 , -0.5904711 , -0.32417855]],
       dtype=float32),
 array([-0.01052125, -0.88623625,  0.6775574 ,  0.15670274, -1.4806497 ],
       dtype=float32),
 array([[ 0.11456871, -0.11503306, -1.3995342 , -2.9320226 , 10.674542  ,
          7.4178996 ,  0.31648788,  0.9708746 , -0.6354385 ,  0.12557863],
        [-0.5933209 ,  0.5133555 ,  0.49433628, -0.11487687,  1.3741077 ,
          1.0474004 ,  0.3452741 , -0.4014675 ,  0.41527912, -0.19326656],
        [-1.8676417 ,  0.7778452 ,  2.1297069 , -4.82994

In [37]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_sigmoid_3rd.h5')

In [38]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_tanh(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 819us/step - loss:

Epoch 68/1000
274/274 [==============================] - 0s 98us/step - loss: 21.4056 - val_loss: 21.8160
Epoch 69/1000
274/274 [==============================] - 0s 98us/step - loss: 21.1816 - val_loss: 21.4652
Epoch 70/1000
274/274 [==============================] - 0s 102us/step - loss: 20.6375 - val_loss: 20.5711
Epoch 71/1000
274/274 [==============================] - 0s 95us/step - loss: 19.7129 - val_loss: 19.3166
Epoch 72/1000
274/274 [==============================] - 0s 94us/step - loss: 17.6425 - val_loss: 16.7103
Epoch 73/1000
274/274 [==============================] - 0s 98us/step - loss: 15.7995 - val_loss: 15.5014
Epoch 74/1000
274/274 [==============================] - 0s 98us/step - loss: 15.0512 - val_loss: 14.8327
Epoch 75/1000
274/274 [==============================] - 0s 95us/step - loss: 13.9182 - val_loss: 14.6684
Epoch 76/1000
274/274 [==============================] - 0s 98us/step - loss: 13.3001 - val_loss: 14.0204
Epoch 77/1000
274/274 [======================

274/274 [==============================] - 0s 109us/step - loss: 6.0749 - val_loss: 10.2261
Epoch 146/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9457 - val_loss: 10.1049
Epoch 147/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0864 - val_loss: 10.0012
Epoch 148/1000
274/274 [==============================] - 0s 106us/step - loss: 6.1263 - val_loss: 10.2163
Epoch 149/1000
274/274 [==============================] - 0s 93us/step - loss: 5.9462 - val_loss: 10.1151
Epoch 150/1000
274/274 [==============================] - 0s 106us/step - loss: 5.9479 - val_loss: 9.9360
Epoch 151/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0305 - val_loss: 9.9647
Epoch 152/1000
274/274 [==============================] - 0s 94us/step - loss: 5.9374 - val_loss: 10.0025
Epoch 153/1000
274/274 [==============================] - 0s 103us/step - loss: 5.9352 - val_loss: 9.7950
Epoch 154/1000
274/274 [==============================] - 0

274/274 [==============================] - 0s 95us/step - loss: 5.3029 - val_loss: 9.0225
Epoch 223/1000
274/274 [==============================] - 0s 109us/step - loss: 5.3491 - val_loss: 9.0642
Epoch 224/1000
274/274 [==============================] - 0s 99us/step - loss: 5.2903 - val_loss: 8.9158
Epoch 225/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3910 - val_loss: 8.7810
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2140 - val_loss: 8.8551
Epoch 227/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3376 - val_loss: 9.0299
Epoch 228/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2966 - val_loss: 8.9465
Epoch 229/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2484 - val_loss: 8.9813
Epoch 230/1000
274/274 [==============================] - 0s 113us/step - loss: 5.3199 - val_loss: 8.9463
Epoch 231/1000
274/274 [==============================] - 0s 120u

274/274 [==============================] - 0s 102us/step - loss: 5.1885 - val_loss: 8.7130
Epoch 300/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1166 - val_loss: 8.6037
Epoch 301/1000
274/274 [==============================] - 0s 97us/step - loss: 5.1225 - val_loss: 8.6622
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1008 - val_loss: 8.6901
Epoch 303/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1800 - val_loss: 8.8606
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1126 - val_loss: 8.8623
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2230 - val_loss: 8.7140
Epoch 306/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2205 - val_loss: 8.5533
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0768 - val_loss: 8.5477
Epoch 308/1000
274/274 [==============================] - 0s 102us/

...

Epoch 760/1000
274/274 [==============================] - 0s 107us/step - loss: 4.3744 - val_loss: 12.3755
Epoch 761/1000
274/274 [==============================] - 0s 127us/step - loss: 4.3156 - val_loss: 12.5066
Epoch 762/1000
274/274 [==============================] - 0s 106us/step - loss: 4.4768 - val_loss: 12.4071
Epoch 763/1000
274/274 [==============================] - 0s 99us/step - loss: 4.5254 - val_loss: 12.5677
Epoch 764/1000
274/274 [==============================] - 0s 98us/step - loss: 4.4263 - val_loss: 12.6711
Epoch 765/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3283 - val_loss: 12.4369
Epoch 766/1000
274/274 [==============================] - 0s 95us/step - loss: 4.3535 - val_loss: 12.6360
Epoch 767/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3573 - val_loss: 12.7197
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 4.4015 - val_loss: 12.6584
Epoch 769/1000
274/274 [=================

Epoch 837/1000
274/274 [==============================] - 0s 106us/step - loss: 4.3472 - val_loss: 12.7692
Epoch 838/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2640 - val_loss: 12.6498
Epoch 839/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2251 - val_loss: 12.7579
Epoch 840/1000
274/274 [==============================] - 0s 95us/step - loss: 4.2668 - val_loss: 12.6033
Epoch 841/1000
274/274 [==============================] - 0s 100us/step - loss: 4.3219 - val_loss: 12.8300
Epoch 842/1000
274/274 [==============================] - 0s 104us/step - loss: 4.3222 - val_loss: 12.5275
Epoch 843/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2955 - val_loss: 12.5155
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 4.3178 - val_loss: 12.6207
Epoch 845/1000
274/274 [==============================] - 0s 101us/step - loss: 4.2900 - val_loss: 12.6298
Epoch 846/1000
274/274 [=================

274/274 [==============================] - 0s 102us/step - loss: 4.3444 - val_loss: 13.1688
Epoch 914/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2652 - val_loss: 12.7605
Epoch 915/1000
274/274 [==============================] - 0s 108us/step - loss: 4.2240 - val_loss: 12.9239
Epoch 916/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2721 - val_loss: 13.2375
Epoch 917/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2804 - val_loss: 12.7837
Epoch 918/1000
274/274 [==============================] - 0s 98us/step - loss: 4.3007 - val_loss: 13.0457
Epoch 919/1000
274/274 [==============================] - 0s 113us/step - loss: 4.4907 - val_loss: 12.8718
Epoch 920/1000
274/274 [==============================] - 0s 106us/step - loss: 4.2398 - val_loss: 12.9879
Epoch 921/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2762 - val_loss: 13.1738
Epoch 922/1000
274/274 [==============================] 

Epoch 990/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3721 - val_loss: 14.8345
Epoch 991/1000
274/274 [==============================] - 0s 95us/step - loss: 4.2508 - val_loss: 14.7737
Epoch 992/1000
274/274 [==============================] - 0s 98us/step - loss: 4.3582 - val_loss: 14.5232
Epoch 993/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3558 - val_loss: 14.3200
Epoch 994/1000
274/274 [==============================] - 0s 95us/step - loss: 4.2923 - val_loss: 14.4334
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2408 - val_loss: 14.7218
Epoch 996/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3351 - val_loss: 14.6640
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 4.4372 - val_loss: 14.5366
Epoch 998/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2554 - val_loss: 15.6062
Epoch 999/1000
274/274 [==================

[array([[-0.61688435, -0.22609827, -0.6342331 ,  0.366641  , -0.9687269 ],
        [-1.5270045 , -1.6959621 ,  0.7699968 ,  2.0398297 ,  0.8230238 ],
        [-2.937295  , -0.8213965 ,  0.57011265, -2.209137  , -1.0982063 ],
        [ 0.9483623 , -0.3457916 ,  0.5764095 , -0.00502201, -3.093615  ],
        [-0.21967639, -0.1502939 ,  1.3470758 ,  0.3034525 ,  0.974901  ],
        [ 0.9363165 ,  1.0671358 , -0.7812648 ,  2.4391727 ,  0.18126158],
        [ 1.444995  , -2.4483917 , -2.198712  ,  1.8460821 ,  1.0266968 ]],
       dtype=float32),
 array([-0.42747882,  0.8547657 ,  0.797431  , -2.1884248 ,  0.02251981],
       dtype=float32),
 array([[ 2.7888947 , -3.6909642 , -1.9547813 , -8.241312  ,  1.260696  ,
         -0.97170186,  0.04239525,  0.3131812 , -1.2136418 ,  1.7288454 ],
        [ 0.8242471 , -0.35555294, -0.5565403 , -0.2569004 ,  8.258198  ,
         -4.1294665 ,  0.5604974 , -8.3181    ,  1.4962405 ,  0.47532576],
        [ 1.1886945 , -0.01838726, -0.67866504,  0.37782

In [39]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_tanh_3rd.h5')

In [40]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_linear(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 1

143/143 [==============================] - 0s 119us/step - loss: 0.0073 - val_loss: 0.0053
Epoch 70/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0070 - val_loss: 0.0052
Epoch 71/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0072 - val_loss: 0.0060
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0070 - val_loss: 0.0048
Epoch 73/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0069 - val_loss: 0.0049
Epoch 74/1000
143/143 [==============================] - ETA: 0s - loss: 0.005 - 0s 117us/step - loss: 0.0070 - val_loss: 0.0050
Epoch 75/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0077 - val_loss: 0.0049
Epoch 76/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0069 - val_loss: 0.0048
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0069 - val_loss: 0.0055
Epoch 78/1000
143/143 [======================

143/143 [==============================] - 0s 112us/step - loss: 0.0058 - val_loss: 0.0044
Epoch 147/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0065 - val_loss: 0.0054
Epoch 148/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0064 - val_loss: 0.0079
Epoch 149/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0070 - val_loss: 0.0055
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0068 - val_loss: 0.0050
Epoch 151/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0057 - val_loss: 0.0053
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0059 - val_loss: 0.0063
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0066 - val_loss: 0.0055
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0047
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0048 - val_loss: 0.0046
Epoch 225/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0045 - val_loss: 0.0045
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0045
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0046 - val_loss: 0.0070
Epoch 228/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0054 - val_loss: 0.0061
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0054 - val_loss: 0.0049
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0051 - val_loss: 0.0049
Epoch 231/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0044
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0039 - val_loss: 0.0045
Epoch 301/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0043
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0044
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0049
Epoch 304/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0040 - val_loss: 0.0048
Epoch 305/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0042 - val_loss: 0.0048
Epoch 306/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0045 - val_loss: 0.0049
Epoch 307/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0041 - val_loss: 0.0046
Epoch 308/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0049 - val_loss: 0.0053
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 126us/step - loss: 0.0032 - val_loss: 0.0046
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0043
Epoch 764/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0032 - val_loss: 0.0044
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0050
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0051
Epoch 767/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0039 - val_loss: 0.0051
Epoch 768/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0037 - val_loss: 0.0046
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0044
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0046
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0044
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0032 - val_loss: 0.0045
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0043
Epoch 842/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0031 - val_loss: 0.0054
Epoch 843/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0035 - val_loss: 0.0046
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0048
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0032 - val_loss: 0.0046
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0044
Epoch 847/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0045
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0029 - val_loss: 0.0042
Epoch 917/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0028 - val_loss: 0.0046
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0046
Epoch 919/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0051
Epoch 920/1000
143/143 [==============================] - 0s 141us/step - loss: 0.0030 - val_loss: 0.0043
Epoch 921/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0030 - val_loss: 0.0053
Epoch 922/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0033 - val_loss: 0.0042
Epoch 923/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0029 - val_loss: 0.0045
Epoch 924/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0028 - val_loss: 0.0043
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0056
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0031 - val_loss: 0.0047
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0057
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0040
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0034 - val_loss: 0.0041
Epoch 998/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0036 - val_loss: 0.0065
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0042
Epoch 1000/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0033 - val_loss: 0.0046
0.0035022867377847433


[array([[-0.08962713, -0.07162376,  0.86272144, -0.8574913 , -0.3859361 ],
        [-0.2578802 , -0.42354757,  0.64654094, -0.81552815, -0.61337024],
        [-0.48494235, -0.42052877,  0.32795647, -0.29053494, -0.5026653 ],
        [-0.04074199,  0.012768  ,  0.4591793 , -0.6373241 , -0.22697626],
        [-0.06616182,  0.58017784,  0.30467945, -0.88342166,  0.09881908],
        [-0.03175436, -0.23892978,  0.12526019, -0.41287512, -0.3238747 ],
        [-0.41729733, -0.05783236,  0.78667265, -0.58108217, -0.6180167 ],
        [ 0.09729291, -0.47072613,  0.0353127 , -0.12659504, -0.47025362],
        [ 0.03944265,  0.21105126,  0.46735513, -0.7920194 , -0.27914068],
        [ 0.37471417, -0.1255387 ,  0.21492209, -0.09676607, -0.7528574 ],
        [-0.19723772, -0.43072805,  0.18906215, -0.17858723,  0.08307343],
        [ 0.36781055,  0.45051748,  0.04334582,  1.4489563 , -0.0701236 ],
        [ 0.53410214, -0.2146546 ,  0.629229  ,  1.2746509 , -0.44058597],
        [ 2.1623554 , -0.

In [41]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_linear_3rd.h5')

In [42]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_relu(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_19 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_20 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 112us/step - loss: 0.0060 - val_loss: 0.0055
Epoch 70/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0058 - val_loss: 0.0055
Epoch 71/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0058 - val_loss: 0.0051
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0060 - val_loss: 0.0052
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0057 - val_loss: 0.0054
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0059 - val_loss: 0.0053
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0054 - val_loss: 0.0046
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0063 - val_loss: 0.0061
Epoch 78/1000
143/143 [==============================] - 0s 119us/ste

143/143 [==============================] - 0s 126us/step - loss: 0.0054 - val_loss: 0.0050
Epoch 147/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0051 - val_loss: 0.0041
Epoch 148/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0041 - val_loss: 0.0059
Epoch 149/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0037
Epoch 150/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0059 - val_loss: 0.0052
Epoch 151/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0048 - val_loss: 0.0052
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0042
Epoch 153/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0048 - val_loss: 0.0049
Epoch 154/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0043 - val_loss: 0.0043
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0048
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0037 - val_loss: 0.0037
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0045
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0037 - val_loss: 0.0039
Epoch 227/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0038 - val_loss: 0.0043
Epoch 228/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0040
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0043
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0039
Epoch 231/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0036 - val_loss: 0.0040
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0038
Epoch 301/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0033 - val_loss: 0.0038
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0037
Epoch 303/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0033 - val_loss: 0.0043
Epoch 304/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0037 - val_loss: 0.0038
Epoch 305/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0039 - val_loss: 0.0043
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0034 - val_loss: 0.0037
Epoch 307/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0036
Epoch 308/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0033 - val_loss: 0.0036
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 139us/step - loss: 0.0051 - val_loss: 0.0037
Epoch 763/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0037 - val_loss: 0.0043
Epoch 764/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0033 - val_loss: 0.0037
Epoch 765/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0030 - val_loss: 0.0037
Epoch 766/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0029 - val_loss: 0.0038
Epoch 767/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0040
Epoch 768/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0027 - val_loss: 0.0040
Epoch 769/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0027 - val_loss: 0.0035
Epoch 770/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0027 - val_loss: 0.0040
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0040
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0038
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0035
Epoch 842/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0026 - val_loss: 0.0038
Epoch 843/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0026 - val_loss: 0.0035
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0040
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0034
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0042
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0031
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0033
Epoch 917/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0029 - val_loss: 0.0041
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0031
Epoch 919/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0040
Epoch 920/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0029 - val_loss: 0.0038
Epoch 921/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0026 - val_loss: 0.0036
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0037
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0033
Epoch 924/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0027 - val_loss: 0.0042
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0031
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0040
Epoch 995/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0028 - val_loss: 0.0032
Epoch 996/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0027 - val_loss: 0.0040
Epoch 997/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0026 - val_loss: 0.0032
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0029 - val_loss: 0.0038
Epoch 999/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0027 - val_loss: 0.0034
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0036
0.01369984820485115


[array([[-0.11434013, -0.23101291,  0.23533805,  0.3012941 , -0.01860092],
        [-0.15715042, -0.20493115,  0.07143531,  0.00684328, -0.5178506 ],
        [-0.27560306, -0.5702419 ,  0.32215753, -0.24127077, -0.26484188],
        [-0.26943636,  0.4254921 ,  0.1678433 ,  0.13394669,  0.08261892],
        [-0.14516959, -0.16543148, -0.40655312, -0.18611835, -0.33007383],
        [ 0.01265472, -0.43502322, -0.09677108,  0.8036989 , -0.40769297],
        [-0.14288017, -0.67743695, -0.45989022, -0.52851385, -0.21921177],
        [-0.42936468, -0.6516936 , -1.3956009 ,  0.06700577, -0.58938974],
        [-0.28068936, -0.46360838,  0.8266075 , -0.01271482, -0.6302666 ],
        [-0.31082898, -0.08831491, -0.4858296 ,  0.693719  , -0.6014654 ],
        [-0.04785466, -0.27757713, -1.3705622 ,  0.00760688, -0.49784967],
        [ 0.34956238,  0.26266977, -1.0774711 , -0.02182257, -0.32097387],
        [ 0.24229142,  0.09338704,  0.33844906, -0.26542825, -0.6417921 ],
        [-0.4693879 , -0.

In [43]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_relu_3rd.h5')

In [44]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_sigmoid(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 118us/step - loss: 0.0382 - val_loss: 0.0253
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0382 - val_loss: 0.0304
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0371 - val_loss: 0.0254
Epoch 72/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0382 - val_loss: 0.0269
Epoch 73/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0378 - val_loss: 0.0312
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0254
Epoch 75/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0376 - val_loss: 0.0252
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0373 - val_loss: 0.0281
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0266
Epoch 78/1000
143/143 [==============================] - 0s 112us/ste

143/143 [==============================] - 0s 112us/step - loss: 0.0377 - val_loss: 0.0265
Epoch 147/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0366 - val_loss: 0.0300
Epoch 148/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0374 - val_loss: 0.0268
Epoch 149/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0368 - val_loss: 0.0259
Epoch 150/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0384 - val_loss: 0.0272
Epoch 151/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0367 - val_loss: 0.0251
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0382 - val_loss: 0.0262
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0384 - val_loss: 0.0298
Epoch 154/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0378 - val_loss: 0.0249
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0363 - val_loss: 0.0251
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0377 - val_loss: 0.0262
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0376 - val_loss: 0.0310
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0377 - val_loss: 0.0251
Epoch 227/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0282
Epoch 228/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0372 - val_loss: 0.0298
Epoch 229/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0370 - val_loss: 0.0261
Epoch 230/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0373 - val_loss: 0.0256
Epoch 231/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0276
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 807us/step - loss: 0.0371 - val_loss: 0.0274
Epoch 301/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0372 - val_loss: 0.0280
Epoch 302/1000
143/143 [==============================] - ETA: 0s - loss: 0.037 - 0s 122us/step - loss: 0.0379 - val_loss: 0.0250
Epoch 303/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0379 - val_loss: 0.0297
Epoch 304/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0376 - val_loss: 0.0257
Epoch 305/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0371 - val_loss: 0.0275
Epoch 306/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0372 - val_loss: 0.0280
Epoch 307/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0371 - val_loss: 0.0279
Epoch 308/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0375 - val_loss: 0.0256
Epoch 309/1000
143/143 [=============

...

143/143 [==============================] - 0s 126us/step - loss: 0.0375 - val_loss: 0.0256
Epoch 763/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0372 - val_loss: 0.0277
Epoch 764/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0270
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0268
Epoch 766/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 767/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0375 - val_loss: 0.0277
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0387 - val_loss: 0.0254
Epoch 769/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0376 - val_loss: 0.0298
Epoch 770/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0372 - val_loss: 0.0265
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 121us/step - loss: 0.0375 - val_loss: 0.0282
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0259
Epoch 841/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0371 - val_loss: 0.0265
Epoch 842/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0371 - val_loss: 0.0272
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0371 - val_loss: 0.0277
Epoch 845/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0368 - val_loss: 0.0256
Epoch 846/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0372 - val_loss: 0.0273
Epoch 847/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0369 - val_loss: 0.0272
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0377 - val_loss: 0.0257
Epoch 917/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0373 - val_loss: 0.0287
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0263
Epoch 919/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0374 - val_loss: 0.0275
Epoch 920/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0375 - val_loss: 0.0255
Epoch 921/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0300
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0375 - val_loss: 0.0276
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0266
Epoch 924/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0368 - val_loss: 0.0282
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 125us/step - loss: 0.0368 - val_loss: 0.0294
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0373 - val_loss: 0.0273
Epoch 995/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0383 - val_loss: 0.0250
Epoch 996/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0371 - val_loss: 0.0280
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0375 - val_loss: 0.0303
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0376 - val_loss: 0.0270
Epoch 999/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0376 - val_loss: 0.0255
Epoch 1000/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0286
0.05475309491157532


[array([[-0.49957633, -0.249079  , -0.38113308, -0.2440201 , -0.5736101 ],
        [ 0.18682317, -0.18590075, -0.3746217 , -0.44289958, -0.5318268 ],
        [-0.13577619, -0.05098948, -0.00992793, -0.72858405, -0.37497455],
        [-0.8757082 ,  0.2818046 , -0.14405487,  0.04650554, -0.05325489],
        [ 0.30687892, -0.3677082 ,  0.3341878 , -0.208308  , -0.28156185],
        [-0.17275247,  0.23256966, -0.11469543, -0.71310455, -0.69733375],
        [ 0.01449761, -0.3541085 , -0.3717688 , -0.33210963, -0.4207859 ],
        [ 0.0297125 , -0.06756297, -0.06007409, -0.05890652, -0.28310886],
        [-0.4354146 , -0.29878822, -0.36236635, -0.23251624,  0.05335467],
        [ 0.13461386, -0.3714432 , -0.35374853, -0.11843456,  0.0636933 ],
        [-0.55530876,  0.45570073, -0.44587117, -0.20625833, -0.7265014 ],
        [-0.58676815, -0.37724912, -0.62073404, -0.6551449 ,  0.02402728],
        [-0.4270095 , -0.28017372, -0.62264234, -0.55438703, -0.23027131],
        [-0.26775998,  0.

In [45]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_sigmoid_3rd.h5')

In [46]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_tanh(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_23 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 112us/step - loss: 0.0371 - val_loss: 0.0279
Epoch 70/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0263
Epoch 71/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0370 - val_loss: 0.0273
Epoch 72/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 74/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0369 - val_loss: 0.0270
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 77/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0370 - val_loss: 0.0269
Epoch 78/1000
143/143 [==============================] - 0s 112us/ste

143/143 [==============================] - 0s 133us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0267
Epoch 148/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 149/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0368 - val_loss: 0.0263
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0262
Epoch 151/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0372 - val_loss: 0.0260
Epoch 152/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0280
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0369 - val_loss: 0.0271
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0273
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 228/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0369 - val_loss: 0.0264
Epoch 229/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0280
Epoch 231/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0369 - val_loss: 0.0288
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0260
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0263
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 303/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0278
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0275
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 306/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0370 - val_loss: 0.0266
Epoch 307/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0273
Epoch 763/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0273
Epoch 766/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0368 - val_loss: 0.0273
Epoch 767/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 768/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0368 - val_loss: 0.0263
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0267
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0266
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0264
Epoch 841/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0369 - val_loss: 0.0262
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0267
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0367 - val_loss: 0.0274
Epoch 844/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0370 - val_loss: 0.0281
Epoch 845/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0370 - val_loss: 0.0282
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0269
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0266
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0281
Epoch 917/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0369 - val_loss: 0.0271
Epoch 918/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0368 - val_loss: 0.0265
Epoch 919/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 921/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 922/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0277
Epoch 923/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0275
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0268
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0275
Epoch 994/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0369 - val_loss: 0.0272
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0266
Epoch 997/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0369 - val_loss: 0.0265
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 999/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0272
0.054677072912454605


[array([[-0.18902512, -0.6961768 , -0.03246891,  0.08734912,  0.01499031],
        [ 0.03572389, -0.47823066,  0.30169538, -0.569627  , -0.71148   ],
        [-0.79726505, -0.06101352,  0.3907086 , -0.58199483, -0.54690045],
        [-0.29450467, -0.31228286, -0.45967826, -0.46479443, -0.05151529],
        [ 0.13206804, -0.45352066, -0.00948709,  0.5802333 , -0.2353901 ],
        [-0.65830445, -0.26036334, -0.22236384, -0.15239865, -0.2998223 ],
        [-0.35576203, -0.70990014, -0.1361886 , -0.06226719, -0.6989994 ],
        [-0.30096236, -0.49809712, -0.3195572 , -0.50653124, -0.6875614 ],
        [ 0.06818407, -0.40636608, -0.22167894, -0.28610733, -0.5927921 ],
        [-0.13852786, -0.2395119 , -0.41353455, -0.12625119, -0.83585775],
        [-0.16287214, -0.12481068, -0.07923281, -0.2977652 , -0.3351205 ],
        [-0.05818196, -0.02373775, -0.27866   ,  0.05934656,  0.09832562],
        [-0.04890495, -0.65499747, -0.25794208, -0.45954183, -0.0926355 ],
        [ 0.07852758, -0.

In [47]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_tanh_3rd.h5')

In [48]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_linear(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_25 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_26 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 9ms/step - loss: 35.565

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0098 - val_loss: 0.0207
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0080 - val_loss: 0.0226
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0072 - val_loss: 0.0248
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0073 - val_loss: 0.0268
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0080 - val_loss: 0.0279
Epoch 76/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0086 - val_loss: 0.0280
Epoch 77/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0089 - val_loss: 0.0268
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0086 - val_loss: 0.0247
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0079 - val_loss: 0.0220
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0027 - val_loss: 0.0060
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0026 - val_loss: 0.0059
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 0.0059
Epoch 154/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0026 - val_loss: 0.0058
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 0.0057
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0026 - val_loss: 0.0057
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0056
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0055
Epoch 159/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0025 - val_loss: 0.0055
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0015 - val_loss: 0.0030
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0015 - val_loss: 0.0030
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0015 - val_loss: 0.0030
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0015 - val_loss: 0.0029
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0029
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0029
Epoch 236/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0014 - val_loss: 0.0029
Epoch 237/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0014 - val_loss: 0.0028
Epoch 238/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0014 - val_loss: 0.0028
Epoch 239/1000
32/32 [==============================] - 0s 156us

32/32 [==============================] - 0s 125us/step - loss: 8.8091e-04 - val_loss: 0.0017
Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 8.7519e-04 - val_loss: 0.0017
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 8.6952e-04 - val_loss: 0.0017
Epoch 311/1000
32/32 [==============================] - 0s 125us/step - loss: 8.6388e-04 - val_loss: 0.0016
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 8.5830e-04 - val_loss: 0.0016
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 8.5273e-04 - val_loss: 0.0016
Epoch 314/1000
32/32 [==============================] - 0s 124us/step - loss: 8.4721e-04 - val_loss: 0.0016
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 8.4174e-04 - val_loss: 0.0016
Epoch 316/1000
32/32 [==============================] - 0s 125us/step - loss: 8.3630e-04 - val_loss: 0.0016
Epoch 317/1000
32/32 [=====================

...

Epoch 754/1000
32/32 [==============================] - 0s 156us/step - loss: 7.9035e-05 - val_loss: 1.1428e-04
Epoch 755/1000
32/32 [==============================] - 0s 155us/step - loss: 7.8745e-05 - val_loss: 1.1377e-04
Epoch 756/1000
32/32 [==============================] - 0s 125us/step - loss: 7.8450e-05 - val_loss: 1.1327e-04
Epoch 757/1000
32/32 [==============================] - 0s 155us/step - loss: 7.8158e-05 - val_loss: 1.1278e-04
Epoch 758/1000
32/32 [==============================] - 0s 156us/step - loss: 7.7872e-05 - val_loss: 1.1229e-04
Epoch 759/1000
32/32 [==============================] - 0s 124us/step - loss: 7.7584e-05 - val_loss: 1.1179e-04
Epoch 760/1000
32/32 [==============================] - 0s 155us/step - loss: 7.7299e-05 - val_loss: 1.1131e-04
Epoch 761/1000
32/32 [==============================] - 0s 125us/step - loss: 7.7017e-05 - val_loss: 1.1083e-04
Epoch 762/1000
32/32 [==============================] - 0s 124us/step - loss: 7.6738e-05 - val_loss: 1.1

Epoch 828/1000
32/32 [==============================] - 0s 124us/step - loss: 6.1660e-05 - val_loss: 8.4591e-05
Epoch 829/1000
32/32 [==============================] - 0s 125us/step - loss: 6.1474e-05 - val_loss: 8.4283e-05
Epoch 830/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1292e-05 - val_loss: 8.3967e-05
Epoch 831/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1110e-05 - val_loss: 8.3664e-05
Epoch 832/1000
32/32 [==============================] - 0s 156us/step - loss: 6.0930e-05 - val_loss: 8.3353e-05
Epoch 833/1000
32/32 [==============================] - 0s 156us/step - loss: 6.0753e-05 - val_loss: 8.3049e-05
Epoch 834/1000
32/32 [==============================] - 0s 156us/step - loss: 6.0572e-05 - val_loss: 8.2745e-05
Epoch 835/1000
32/32 [==============================] - 0s 125us/step - loss: 6.0398e-05 - val_loss: 8.2440e-05
Epoch 836/1000
32/32 [==============================] - 0s 156us/step - loss: 6.0223e-05 - val_loss: 8.2

Epoch 902/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0721e-05 - val_loss: 6.5892e-05
Epoch 903/1000
32/32 [==============================] - 0s 187us/step - loss: 5.0605e-05 - val_loss: 6.5692e-05
Epoch 904/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0490e-05 - val_loss: 6.5493e-05
Epoch 905/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0372e-05 - val_loss: 6.5302e-05
Epoch 906/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0259e-05 - val_loss: 6.5097e-05
Epoch 907/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0144e-05 - val_loss: 6.4905e-05
Epoch 908/1000
32/32 [==============================] - 0s 155us/step - loss: 5.0032e-05 - val_loss: 6.4713e-05
Epoch 909/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9918e-05 - val_loss: 6.4521e-05
Epoch 910/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9807e-05 - val_loss: 6.4

Epoch 976/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3644e-05 - val_loss: 5.3746e-05
Epoch 977/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3566e-05 - val_loss: 5.3612e-05
Epoch 978/1000
32/32 [==============================] - 0s 125us/step - loss: 4.3488e-05 - val_loss: 5.3477e-05
Epoch 979/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3413e-05 - val_loss: 5.3347e-05
Epoch 980/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3339e-05 - val_loss: 5.3215e-05
Epoch 981/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3262e-05 - val_loss: 5.3087e-05
Epoch 982/1000
32/32 [==============================] - 0s 187us/step - loss: 4.3187e-05 - val_loss: 5.2959e-05
Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3111e-05 - val_loss: 5.2827e-05
Epoch 984/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3037e-05 - val_loss: 5.2

[array([[-0.10643508,  0.54745764, -0.05210108,  0.4669482 ,  0.4391452 ],
        [-1.84319   , -0.30426893, -0.62738734, -0.4054012 , -1.3709867 ],
        [ 0.5649113 , -0.28606787,  1.9581379 , -0.26380616,  1.0169952 ]],
       dtype=float32),
 array([-0.7721934 ,  0.75424886,  0.7743762 ,  0.54508924, -0.8148524 ],
       dtype=float32),
 array([[ 0.14334433,  0.15444417,  0.22191133,  0.42854095,  0.13896328,
          0.22103484, -0.14986765, -0.35678798,  0.40056825,  0.458663  ],
        [-0.06148617, -0.46502087, -0.07358157,  0.04440384, -0.07494152,
         -0.7138819 ,  0.47739705,  0.44126603,  0.6834026 ,  0.28745282],
        [-0.43262243,  0.546841  ,  0.23919335, -0.25790882,  0.43518898,
          0.17378587, -0.04364296, -0.40937567, -0.44869474,  0.34150118],
        [-0.30034098, -0.13103005,  0.3807028 ,  0.40733862,  0.06492361,
         -0.9796437 ,  0.74423414,  0.16784516, -0.5434292 , -0.25296772],
        [ 0.03275327, -0.01874728, -0.09551244,  0.4966205

In [49]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_linear_3rd.h5')

In [50]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_relu(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_27 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_28 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 9ms/step - loss: 38.187

Epoch 71/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0384 - val_loss: 0.0390
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0366 - val_loss: 0.0384
Epoch 73/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0345 - val_loss: 0.0380
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0325 - val_loss: 0.0378
Epoch 75/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0306 - val_loss: 0.0377
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0290 - val_loss: 0.0376
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0277 - val_loss: 0.0375
Epoch 78/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0267 - val_loss: 0.0373
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0259 - val_loss: 0.0368
Epoch 80/1000
32/32 [==============================] - 0s 173us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0060 - val_loss: 0.0055
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0059 - val_loss: 0.0054
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0058 - val_loss: 0.0053
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0057 - val_loss: 0.0052
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0057 - val_loss: 0.0051
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0056 - val_loss: 0.0049
Epoch 157/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 158/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 159/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 160/1000
32/32 [==============================] - 0s 134us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 233/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0012
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0012
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0012
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0012
Epoch 238/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 0.0012
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 306/1000
32/32 [==============================] - 0s 125us/step - loss: 6.6825e-04 - val_loss: 4.0837e-04
Epoch 307/1000
32/32 [==============================] - 0s 151us/step - loss: 6.5883e-04 - val_loss: 4.0226e-04
Epoch 308/1000
32/32 [==============================] - 0s 156us/step - loss: 6.4953e-04 - val_loss: 3.9624e-04
Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 6.4035e-04 - val_loss: 3.9033e-04
Epoch 310/1000
32/32 [==============================] - 0s 124us/step - loss: 6.3131e-04 - val_loss: 3.8450e-04
Epoch 311/1000
32/32 [==============================] - 0s 187us/step - loss: 6.2238e-04 - val_loss: 3.7875e-04
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1357e-04 - val_loss: 3.7310e-04
Epoch 313/1000
32/32 [==============================] - 0s 125us/step - loss: 6.0489e-04 - val_loss: 3.6753e-04
Epoch 314/1000
32/32 [==============================] - 0s 155us/step - loss: 5.9632e-04 - val_loss: 3.6

...

Epoch 750/1000
32/32 [==============================] - 0s 125us/step - loss: 4.7454e-07 - val_loss: 2.6742e-07
Epoch 751/1000
32/32 [==============================] - 0s 157us/step - loss: 4.6575e-07 - val_loss: 2.6233e-07
Epoch 752/1000
32/32 [==============================] - 0s 155us/step - loss: 4.5715e-07 - val_loss: 2.5734e-07
Epoch 753/1000
32/32 [==============================] - 0s 125us/step - loss: 4.4876e-07 - val_loss: 2.5265e-07
Epoch 754/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4055e-07 - val_loss: 2.4793e-07
Epoch 755/1000
32/32 [==============================] - 0s 187us/step - loss: 4.3234e-07 - val_loss: 2.4333e-07
Epoch 756/1000
32/32 [==============================] - 0s 156us/step - loss: 4.2441e-07 - val_loss: 2.3883e-07
Epoch 757/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1655e-07 - val_loss: 2.3451e-07
Epoch 758/1000
32/32 [==============================] - 0s 125us/step - loss: 4.0894e-07 - val_loss: 2.3

Epoch 824/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1677e-07 - val_loss: 6.5557e-08
Epoch 825/1000
32/32 [==============================] - 0s 125us/step - loss: 1.1448e-07 - val_loss: 6.4378e-08
Epoch 826/1000
32/32 [==============================] - 0s 125us/step - loss: 1.1228e-07 - val_loss: 6.2978e-08
Epoch 827/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1010e-07 - val_loss: 6.1880e-08
Epoch 828/1000
32/32 [==============================] - 0s 124us/step - loss: 1.0795e-07 - val_loss: 6.0590e-08
Epoch 829/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0581e-07 - val_loss: 5.9599e-08
Epoch 830/1000
32/32 [==============================] - 0s 155us/step - loss: 1.0386e-07 - val_loss: 5.8451e-08
Epoch 831/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0181e-07 - val_loss: 5.7377e-08
Epoch 832/1000
32/32 [==============================] - 0s 155us/step - loss: 9.9916e-08 - val_loss: 5.6

Epoch 898/1000
32/32 [==============================] - 0s 125us/step - loss: 2.7006e-08 - val_loss: 1.5244e-08
Epoch 899/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6468e-08 - val_loss: 1.4922e-08
Epoch 900/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5973e-08 - val_loss: 1.4598e-08
Epoch 901/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5430e-08 - val_loss: 1.4258e-08
Epoch 902/1000
32/32 [==============================] - 0s 125us/step - loss: 2.4931e-08 - val_loss: 1.3977e-08
Epoch 903/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4435e-08 - val_loss: 1.3695e-08
Epoch 904/1000
32/32 [==============================] - 0s 125us/step - loss: 2.3954e-08 - val_loss: 1.3448e-08
Epoch 905/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3481e-08 - val_loss: 1.3154e-08
Epoch 906/1000
32/32 [==============================] - 0s 124us/step - loss: 2.3023e-08 - val_loss: 1.2

Epoch 972/1000
32/32 [==============================] - 0s 156us/step - loss: 5.8940e-09 - val_loss: 3.3389e-09
Epoch 973/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7644e-09 - val_loss: 3.2617e-09
Epoch 974/1000
32/32 [==============================] - 0s 155us/step - loss: 5.6293e-09 - val_loss: 3.1573e-09
Epoch 975/1000
32/32 [==============================] - 0s 157us/step - loss: 5.5259e-09 - val_loss: 3.1149e-09
Epoch 976/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4223e-09 - val_loss: 3.0189e-09
Epoch 977/1000
32/32 [==============================] - 0s 156us/step - loss: 5.2951e-09 - val_loss: 2.9701e-09
Epoch 978/1000
32/32 [==============================] - 0s 156us/step - loss: 5.1890e-09 - val_loss: 2.8942e-09
Epoch 979/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0742e-09 - val_loss: 2.8622e-09
Epoch 980/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9701e-09 - val_loss: 2.8

[array([[-5.0512969e-01, -4.6921757e-04, -2.0486562e-02, -7.8703612e-01,
          1.0930594e+00],
        [-7.3067862e-01, -1.8316913e-01,  8.1406611e-01,  6.4535424e-02,
         -1.4243491e-01],
        [-4.0648007e-01, -3.5018343e-01,  8.9940965e-01, -2.0395689e-01,
          1.8476747e-01]], dtype=float32),
 array([ 0.03973232, -0.87257665, -0.11280363,  0.3913937 ,  0.47099224],
       dtype=float32),
 array([[-0.01115614, -0.2148926 , -0.17948215, -0.9198766 , -0.7324524 ,
         -0.4067296 ,  0.48221585, -0.1654384 , -0.64242756, -0.5671849 ],
        [ 0.7274891 ,  0.05872794, -0.1598261 , -0.3036224 , -0.6564543 ,
         -0.06771255, -0.00233848,  0.17615391, -0.25959533, -0.6724642 ],
        [ 0.25946453, -0.2461972 , -0.14643149, -0.56728023, -0.5555576 ,
         -0.30541337,  0.1951243 , -0.6604656 , -0.22110936, -0.80494845],
        [ 0.3896581 , -0.62934005, -0.6545125 , -0.17608666, -0.10321164,
         -0.6887154 ,  0.5622161 , -0.5420234 , -0.7465959 ,  0.0148

In [51]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_relu_3rd.h5')

In [52]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_sigmoid(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_29 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_30 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 38.76

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0078 - val_loss: 0.0045
Epoch 72/1000
32/32 [==============================] - 0s 176us/step - loss: 0.0062 - val_loss: 0.0032
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0048 - val_loss: 0.0021
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0036 - val_loss: 0.0013
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 8.0921e-04
Epoch 76/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0020 - val_loss: 5.1357e-04
Epoch 77/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0016 - val_loss: 4.2624e-04
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0015 - val_loss: 5.0228e-04
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0015 - val_loss: 6.9206e-04
Epoch 80/1000
32/32 [==============================] 

Epoch 147/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3932e-04 - val_loss: 1.0725e-04
Epoch 148/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3353e-04 - val_loss: 1.0533e-04
Epoch 149/1000
32/32 [==============================] - 0s 125us/step - loss: 4.2775e-04 - val_loss: 1.0343e-04
Epoch 150/1000
32/32 [==============================] - 0s 125us/step - loss: 4.2199e-04 - val_loss: 1.0152e-04
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1626e-04 - val_loss: 9.9635e-05
Epoch 152/1000
32/32 [==============================] - 0s 155us/step - loss: 4.1060e-04 - val_loss: 9.7766e-05
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0500e-04 - val_loss: 9.5940e-05
Epoch 154/1000
32/32 [==============================] - 0s 187us/step - loss: 3.9951e-04 - val_loss: 9.4169e-05
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9415e-04 - val_loss: 9.2

Epoch 221/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9207e-04 - val_loss: 3.6011e-05
Epoch 222/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9033e-04 - val_loss: 3.5626e-05
Epoch 223/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8862e-04 - val_loss: 3.5251e-05
Epoch 224/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8694e-04 - val_loss: 3.4878e-05
Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8528e-04 - val_loss: 3.4517e-05
Epoch 226/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8364e-04 - val_loss: 3.4162e-05
Epoch 227/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8203e-04 - val_loss: 3.3812e-05
Epoch 228/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8043e-04 - val_loss: 3.3471e-05
Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7886e-04 - val_loss: 3.3

Epoch 295/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1056e-04 - val_loss: 2.2614e-05
Epoch 296/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0991e-04 - val_loss: 2.2569e-05
Epoch 297/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0926e-04 - val_loss: 2.2526e-05
Epoch 298/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0862e-04 - val_loss: 2.2485e-05
Epoch 299/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0799e-04 - val_loss: 2.2445e-05
Epoch 300/1000
32/32 [==============================] - 0s 187us/step - loss: 1.0736e-04 - val_loss: 2.2410e-05
Epoch 301/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0675e-04 - val_loss: 2.2372e-05
Epoch 302/1000
32/32 [==============================] - 0s 124us/step - loss: 1.0614e-04 - val_loss: 2.2337e-05
Epoch 303/1000
32/32 [==============================] - 0s 124us/step - loss: 1.0554e-04 - val_loss: 2.2

...

Epoch 739/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4146e-05 - val_loss: 2.1784e-05
Epoch 740/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4111e-05 - val_loss: 2.1763e-05
Epoch 741/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4076e-05 - val_loss: 2.1739e-05
Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4040e-05 - val_loss: 2.1716e-05
Epoch 743/1000
32/32 [==============================] - 0s 155us/step - loss: 4.4005e-05 - val_loss: 2.1692e-05
Epoch 744/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3970e-05 - val_loss: 2.1666e-05
Epoch 745/1000
32/32 [==============================] - 0s 125us/step - loss: 4.3937e-05 - val_loss: 2.1641e-05
Epoch 746/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3900e-05 - val_loss: 2.1618e-05
Epoch 747/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3865e-05 - val_loss: 2.1

Epoch 813/1000
32/32 [==============================] - 0s 157us/step - loss: 4.1665e-05 - val_loss: 1.9991e-05
Epoch 814/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1633e-05 - val_loss: 1.9965e-05
Epoch 815/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1601e-05 - val_loss: 1.9944e-05
Epoch 816/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1569e-05 - val_loss: 1.9918e-05
Epoch 817/1000
32/32 [==============================] - 0s 157us/step - loss: 4.1538e-05 - val_loss: 1.9896e-05
Epoch 818/1000
32/32 [==============================] - 0s 155us/step - loss: 4.1506e-05 - val_loss: 1.9870e-05
Epoch 819/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1474e-05 - val_loss: 1.9846e-05
Epoch 820/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1443e-05 - val_loss: 1.9820e-05
Epoch 821/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1410e-05 - val_loss: 1.9

Epoch 887/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9404e-05 - val_loss: 1.8245e-05
Epoch 888/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9372e-05 - val_loss: 1.8225e-05
Epoch 889/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9344e-05 - val_loss: 1.8200e-05
Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9315e-05 - val_loss: 1.8179e-05
Epoch 891/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9286e-05 - val_loss: 1.8157e-05
Epoch 892/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9256e-05 - val_loss: 1.8133e-05
Epoch 893/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9228e-05 - val_loss: 1.8111e-05
Epoch 894/1000
32/32 [==============================] - 0s 125us/step - loss: 3.9196e-05 - val_loss: 1.8089e-05
Epoch 895/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9168e-05 - val_loss: 1.8

Epoch 961/1000
32/32 [==============================] - 0s 125us/step - loss: 3.7301e-05 - val_loss: 1.6641e-05
Epoch 962/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7275e-05 - val_loss: 1.6621e-05
Epoch 963/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7245e-05 - val_loss: 1.6600e-05
Epoch 964/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7219e-05 - val_loss: 1.6582e-05
Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7192e-05 - val_loss: 1.6559e-05
Epoch 966/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7164e-05 - val_loss: 1.6540e-05
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7138e-05 - val_loss: 1.6521e-05
Epoch 968/1000
32/32 [==============================] - 0s 125us/step - loss: 3.7109e-05 - val_loss: 1.6501e-05
Epoch 969/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7084e-05 - val_loss: 1.6

[array([[ 0.6535375 , -1.3886149 , -1.1965309 ,  0.26288807, -1.4442126 ],
        [-0.46074045,  1.8943005 ,  2.036026  ,  0.42521158,  1.7579416 ],
        [ 0.9220285 , -1.3722011 , -1.3530482 , -0.6036215 , -1.6836631 ]],
       dtype=float32),
 array([ 0.13367583,  1.2217376 ,  1.1613909 , -0.9290095 ,  1.1297528 ],
       dtype=float32),
 array([[ 0.98176455,  1.3423132 , -0.67828274,  0.1312533 ,  0.25808567,
         -0.31067145,  1.3317345 ,  0.69519466, -0.5983292 , -0.93270844],
        [ 0.8551645 ,  0.7156056 , -0.48553753,  1.1200254 ,  0.32801774,
         -0.5242196 ,  0.48911062,  1.6404837 , -1.0881591 , -1.2810957 ],
        [ 1.1387489 ,  1.3260375 , -0.80493057,  0.84101605,  0.86357874,
         -1.2366945 ,  1.1583375 ,  1.0931104 , -1.0199695 , -0.73325634],
        [ 0.41396034,  1.2593446 , -0.6614718 ,  0.41990593, -0.5334636 ,
         -1.2404528 ,  0.2839458 ,  1.2886219 , -0.66249245, -0.2529815 ],
        [ 1.256968  ,  1.1161442 , -1.512835  ,  0.9188146

In [53]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_sigmoid_3rd.h5')

In [54]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_tanh(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_31 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_47 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_32 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 37.51

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 4.6576e-04
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 7.2836e-04
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0020 - val_loss: 0.0013
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0030 - val_loss: 0.0020
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 0.0024
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0045 - val_loss: 0.0025
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0047 - val_loss: 0.0023
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0044 - val_loss: 0.0018
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 0.0013
Epoch 80/1000
32/32 [==============================] - 0s 156us/s

Epoch 147/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3715e-04 - val_loss: 5.0328e-04
Epoch 148/1000
32/32 [==============================] - 0s 187us/step - loss: 7.3420e-04 - val_loss: 4.9484e-04
Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3148e-04 - val_loss: 4.8815e-04
Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 7.2882e-04 - val_loss: 4.8326e-04
Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 7.2615e-04 - val_loss: 4.8011e-04
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 7.2337e-04 - val_loss: 4.7858e-04
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 7.2044e-04 - val_loss: 4.7849e-04
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1738e-04 - val_loss: 4.7964e-04
Epoch 155/1000
32/32 [==============================] - 0s 187us/step - loss: 7.1423e-04 - val_loss: 4.8

Epoch 221/1000
32/32 [==============================] - 0s 249us/step - loss: 5.5934e-04 - val_loss: 4.2128e-04
Epoch 222/1000
32/32 [==============================] - 0s 249us/step - loss: 5.5747e-04 - val_loss: 4.2027e-04
Epoch 223/1000
32/32 [==============================] - 0s 249us/step - loss: 5.5560e-04 - val_loss: 4.1930e-04
Epoch 224/1000
32/32 [==============================] - 0s 218us/step - loss: 5.5376e-04 - val_loss: 4.1834e-04
Epoch 225/1000
32/32 [==============================] - 0s 249us/step - loss: 5.5192e-04 - val_loss: 4.1742e-04
Epoch 226/1000
32/32 [==============================] - 0s 218us/step - loss: 5.5009e-04 - val_loss: 4.1655e-04
Epoch 227/1000
32/32 [==============================] - 0s 187us/step - loss: 5.4827e-04 - val_loss: 4.1569e-04
Epoch 228/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4646e-04 - val_loss: 4.1489e-04
Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4467e-04 - val_loss: 4.1

Epoch 295/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4586e-04 - val_loss: 3.6891e-04
Epoch 296/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4461e-04 - val_loss: 3.6836e-04
Epoch 297/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4337e-04 - val_loss: 3.6780e-04
Epoch 298/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4214e-04 - val_loss: 3.6726e-04
Epoch 299/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4091e-04 - val_loss: 3.6671e-04
Epoch 300/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3968e-04 - val_loss: 3.6617e-04
Epoch 301/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3847e-04 - val_loss: 3.6562e-04
Epoch 302/1000
32/32 [==============================] - 0s 125us/step - loss: 4.3725e-04 - val_loss: 3.6509e-04
Epoch 303/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3606e-04 - val_loss: 3.6

...

Epoch 739/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8101e-04 - val_loss: 2.3156e-04
Epoch 740/1000
32/32 [==============================] - 0s 155us/step - loss: 1.8073e-04 - val_loss: 2.3134e-04
Epoch 741/1000
32/32 [==============================] - 0s 187us/step - loss: 1.8046e-04 - val_loss: 2.3112e-04
Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8018e-04 - val_loss: 2.3091e-04
Epoch 743/1000
32/32 [==============================] - 0s 155us/step - loss: 1.7991e-04 - val_loss: 2.3069e-04
Epoch 744/1000
32/32 [==============================] - 0s 157us/step - loss: 1.7964e-04 - val_loss: 2.3047e-04
Epoch 745/1000
32/32 [==============================] - 0s 155us/step - loss: 1.7937e-04 - val_loss: 2.3026e-04
Epoch 746/1000
32/32 [==============================] - 0s 125us/step - loss: 1.7909e-04 - val_loss: 2.3004e-04
Epoch 747/1000
32/32 [==============================] - 0s 125us/step - loss: 1.7882e-04 - val_loss: 2.2

Epoch 813/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6222e-04 - val_loss: 2.1558e-04
Epoch 814/1000
32/32 [==============================] - 0s 155us/step - loss: 1.6198e-04 - val_loss: 2.1536e-04
Epoch 815/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6175e-04 - val_loss: 2.1515e-04
Epoch 816/1000
32/32 [==============================] - 0s 125us/step - loss: 1.6152e-04 - val_loss: 2.1493e-04
Epoch 817/1000
32/32 [==============================] - 0s 155us/step - loss: 1.6128e-04 - val_loss: 2.1472e-04
Epoch 818/1000
32/32 [==============================] - 0s 157us/step - loss: 1.6105e-04 - val_loss: 2.1450e-04
Epoch 819/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6082e-04 - val_loss: 2.1430e-04
Epoch 820/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6060e-04 - val_loss: 2.1409e-04
Epoch 821/1000
32/32 [==============================] - 0s 125us/step - loss: 1.6036e-04 - val_loss: 2.1

Epoch 887/1000
32/32 [==============================] - 0s 155us/step - loss: 1.4618e-04 - val_loss: 1.9986e-04
Epoch 888/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4598e-04 - val_loss: 1.9965e-04
Epoch 889/1000
32/32 [==============================] - 0s 187us/step - loss: 1.4578e-04 - val_loss: 1.9945e-04
Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4558e-04 - val_loss: 1.9924e-04
Epoch 891/1000
32/32 [==============================] - 0s 155us/step - loss: 1.4538e-04 - val_loss: 1.9903e-04
Epoch 892/1000
32/32 [==============================] - 0s 187us/step - loss: 1.4518e-04 - val_loss: 1.9882e-04
Epoch 893/1000
32/32 [==============================] - 0s 125us/step - loss: 1.4498e-04 - val_loss: 1.9861e-04
Epoch 894/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4478e-04 - val_loss: 1.9840e-04
Epoch 895/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4458e-04 - val_loss: 1.9

Epoch 961/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3231e-04 - val_loss: 1.8449e-04
Epoch 962/1000
32/32 [==============================] - 0s 125us/step - loss: 1.3213e-04 - val_loss: 1.8428e-04
Epoch 963/1000
32/32 [==============================] - 0s 125us/step - loss: 1.3196e-04 - val_loss: 1.8408e-04
Epoch 964/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3178e-04 - val_loss: 1.8388e-04
Epoch 965/1000
32/32 [==============================] - 0s 155us/step - loss: 1.3161e-04 - val_loss: 1.8367e-04
Epoch 966/1000
32/32 [==============================] - 0s 125us/step - loss: 1.3144e-04 - val_loss: 1.8346e-04
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3126e-04 - val_loss: 1.8326e-04
Epoch 968/1000
32/32 [==============================] - 0s 155us/step - loss: 1.3109e-04 - val_loss: 1.8305e-04
Epoch 969/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3091e-04 - val_loss: 1.8

[array([[ 1.0465508e+00, -1.4650474e+00,  1.4114271e-01, -4.4207135e-01,
          8.9004320e-01],
        [-1.2652401e+00,  1.1972096e+00,  3.6841333e-01,  4.4438806e-01,
          3.1295884e-01],
        [-1.3317440e+00,  1.3738480e+00,  3.4802479e-01, -1.3916385e-01,
         -2.0155884e-04]], dtype=float32),
 array([ 0.89825976,  0.88887954, -0.5107071 , -0.22366881, -0.04528725],
       dtype=float32),
 array([[-0.76697904, -1.448889  , -1.0987976 , -0.9460639 , -0.34702334,
          1.0991843 , -0.7522915 , -0.16459979, -1.180037  , -1.4943328 ],
        [-0.30475706, -0.34500203, -0.8159195 , -0.5207732 , -1.000115  ,
          1.159038  , -0.79182225, -0.9740061 , -1.0370418 , -0.79821754],
        [-0.19086751, -0.29096138, -1.0594194 , -0.73945284, -0.88356906,
          0.29974064, -0.871569  , -0.7992014 , -0.13457952, -0.8283284 ],
        [-0.97150177, -1.1779838 , -0.26703128, -0.5569972 , -0.68908614,
          0.23136744, -0.3791112 , -0.01633957, -0.89030725, -1.0521

In [55]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_tanh_3rd.h5')